In [ ]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
import requests
import json


class CustomLLM(LLM):
    max_token: int
    URL: str = "http://xxxxx"
    headers: dict = {"Content-Type": "application/json"}
    payload: dict = {"prompt": "", "history": []}
    logger: Any

    @property
    def _llm_type(self) -> str:
        return "CustomLLM"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        history: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        self.payload["prompt"] = prompt
        response = requests.post(
            self.URL, headers=self.headers, data=json.dumps(self.payload)
        )

        if response.status_code == 200:
            result = response.json()
            return result["response"]
        else:
            self.logger.error("CustomLLM error occurred with status code:", response.text)
        return response.text

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "max_token": self.max_token,
            "URL": self.URL,
            "headers": self.headers,
            "payload": self.payload,
        }

In [ ]:
# import cv2
# import base64
# import json
import logging
from typing import Any, Dict, Optional, List, Tuple
# import re
# import itertools
# import random
from yacs.config import CfgNode
import os
from langchain.chat_models import ChatOpenAI
import math
# from langchain.llms import OpenAI

# logger
def set_logger(log_file, name="default"):
    """
    Set logger.
    Args:
        log_file (str): log file path
        name (str): logger name
    """

    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter(
        "%(asctime)s - %(levelname)s - %(module)s - %(funcName)s - %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )
    print(formatter.format(logging.LogRecord(
    name='root',
    level=logging.INFO,
    pathname=None,
    lineno=None,
    msg='Test message',
    args=None,
    exc_info=None,
    )))
    output_folder = "output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Create the 'log' folder if it doesn't exist
    log_folder = os.path.join(output_folder, "log")
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)

    # Create the 'message' folder if it doesn't exist
    message_folder = os.path.join(output_folder, "message")
    if not os.path.exists(message_folder):
        os.makedirs(message_folder)
    log_file = os.path.join(log_folder, log_file)
    handler = logging.FileHandler(log_file, mode="w")
    handler.setLevel(logging.INFO)
    handler.setFormatter(formatter)
    logger.handlers = []
    logger.addHandler(handler)
    print(logger)
    return logger

def load_cfg(cfg_file: str, new_allowed: bool = True) -> CfgNode:
    """
    Load config from file.
    Args:
        cfg_file (str): config file path
        new_allowed (bool): whether to allow new keys in config
    """
    with open(cfg_file, "r") as fi:
        cfg = CfgNode.load_cfg(fi)
    cfg.set_new_allowed(new_allowed)
    return cfg

def add_variable_to_config(cfg: CfgNode, name: str, value: Any) -> CfgNode:
    """
    Add variable to config.
    Args:
        cfg (CfgNode): config
        name (str): variable name
        value (Any): variable value
    """
    cfg.defrost()
    cfg[name] = value
    cfg.freeze()
    return cfg

def get_llm(config, logger, api_key):
    """
    Get the large language model.
    Args:
        config (CfgNode): The config.
        logger (Logger): The logger.
        api_key (str): The API key.
    """
    if config["llm"] == "gpt-4":
        LLM = ChatOpenAI(
            max_tokens=config["max_token"],
            temperature=config["temperature"],
            openai_api_key=api_key,
            model="gpt-4",
            max_retries=config["max_retries"]
        )
    elif config["llm"] == "gpt-3.5-16k":
        LLM = ChatOpenAI(
            max_tokens=config["max_token"],
            temperature=config["temperature"],
            openai_api_key=api_key,
            model="gpt-3.5-turbo-16k",
            max_retries=config["max_retries"]
        )
    elif config["llm"] == "gpt-3.5":
        LLM = ChatOpenAI(
            max_tokens=config["max_token"],
            temperature=config["temperature"],
            openai_api_key=api_key,
            model="gpt-3.5-turbo",
            max_retries=config["max_retries"]
        )
    elif config["llm"] == "gpt-4o-mini":
        LLM = ChatOpenAI(
            max_tokens=config["max_token"],
            temperature=config["temperature"],
            openai_api_key=api_key,
            model="gpt-4o-mini",
            max_retries=config["max_retries"]
            )
    elif config["llm"] == "gpt-4.1-mini":
        LLM = ChatOpenAI(
            max_tokens=config["max_token"],
            temperature=config["temperature"],
            openai_api_key=api_key,
            model="gpt-4.1-mini",
            max_retries=config["max_retries"]
            )
    elif config["llm"] == "custom":
        LLM = CustomLLM(max_token=2048, logger=logger,max_retries=config["max_retries"])
    elif config["llm"] == "new":
        LLM = OpenAI(model="davinci-002")
    return LLM

def ensure_dir(dir_path):
    """
    Make sure the directory exists, if it does not exist, create it
    Args:
        dir_path (str): The directory path.
    """
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


def generate_id(dir_name):
    ensure_dir(dir_name)
    existed_id = set()
    for f in os.listdir(dir_name):
        existed_id.add(f.split("-")[0])
    id = random.randint(1, 999999999)
    while id in existed_id:
        id = random.randint(1, 999999999)
    return id

def get_feature_description(feature):
    """Get description of given features."""
    descriptions = {
        "Watcher": "Choose movies, enjoy watching, and provide feedback and ratings to the recommendation system.",
        "Explorer": "Search for movies heard of before and expand movie experiences.",
        "Critic": "Demanding high standards for movies and the recommendation system, may criticize both the recommendation system and the movies.",
        "Chatter": "Engage in private conversations, trust friends' recommendations.",
        "Poster": "Enjoy publicly posting on social media and sharing content and insights with more people."
    }
    features = feature.split(";")
    descriptions_list = [descriptions[feature] for feature in features if feature in descriptions]
    return ".".join(descriptions_list)

def extract_item_names(observation: str, action: str = "RECOMMENDER") -> List[str]:
    """
    Extract item names from observation
    Args:
        observation: observation from the environment
        action: action type, RECOMMENDER or SOCIAL
    """
    item_names = []
    if observation.find("<") != -1:
        matches = re.findall(r"<(.*?)>", observation)
        item_names = []
        for match in matches:
            item_names.append(match)
    elif observation.find(";") != -1:
        item_names = observation.split(";")
        item_names = [item.strip(" '\"") for item in item_names]
    elif action == "RECOMMENDER":
        matches = re.findall(r'"([^"]+)"', observation)
        for match in matches:
            item_names.append(match)
    elif action == "SOCIAL":
        matches = re.findall(r'[<"]([^<>"]+)[">]', observation)
        for match in matches:
            item_names.append(match)
    return item_names


def count_files_in_directory(target_directory:str):
    """Count the number of files in the target directory"""
    return len(os.listdir(target_directory))

def get_avatar_url(id:int,gender:str,type:str="origin",role=False):
    if role:
        target='/asset/img/avatar/role/'+gender+'/'
        return target+str(id%10)+'.png'
    target='/asset/img/avatar/'+type+"/"+gender+'/'
    return target+str(id%10)+'.png'

def is_chatting(agent, agent2):
    """Determine if agent1 and agent2 is chatting"""
    name = agent.name
    agent_name2 = agent2.name
    return (
        (agent2.event.target_agent)
        and (agent.event.target_agent)
        and (name in agent2.event.target_agent)
        and (agent_name2 in agent.event.target_agent)
    )

def calculate_entropy(movie_types):
    type_freq = {}
    for movie_type in movie_types:
        if movie_type in type_freq:
            type_freq[movie_type] += 1
        else:
            type_freq[movie_type] = 1

    total_movies = len(movie_types)

    entropy = 0
    for key in type_freq:
        prob = type_freq[key] / total_movies
        entropy -= prob * math.log2(prob)

    return entropy


def get_entropy(inters, data):
    genres = data.get_genres_by_id(inters)
    entropy = calculate_entropy(genres)
    return entropy

In [ ]:
from datetime import datetime, timedelta
from typing import Optional, List
from pydantic import BaseModel
class Event(BaseModel):
    """
    Event class for each agent.
    """
    start_time: datetime
    # action start time
    duration: int
    # action duration. The default unit is hour.
    target_agent: Optional[List[str]]
    # target agent for chatting.
    action_type: str
    # ('watching','chatting','posting' or 'idle').
    end_time: Optional[datetime]=None
    # action end time

    """Event for each agent"""
    def __init__(self, **data):
        super().__init__(**data)
        self.end_time = self.start_time + timedelta(hours=self.duration)


def update_event(original_event, start_time, duration, target_agent, action_type):
    if action_type == "reading" or action_type=="posting":
        result = Event(start_time=start_time, duration=duration, target_agent=None, action_type=action_type)

    if action_type == "chatting":
        # If the agent is not chatting
        if original_event.action_type == "idle" or original_event.action_type == "posting":
            result = Event(start_time=start_time, duration=duration, target_agent=[target_agent], action_type=action_type)
        # If the agent is chatting
        else:
            # maintain maximum chat time
            end_time = start_time + timedelta(hours=duration)  # current chat end time
            original_end_time = original_event.end_time
            if end_time > original_end_time:
                if target_agent in original_event.target_agent:
                    new_target_agent = original_event.target_agent
                else:
                    new_target_agent = original_event.target_agent + [target_agent]
                result = Event(start_time=start_time, duration=duration, target_agent=new_target_agent, action_type=action_type)
            else:
                result = original_event
                if target_agent not in result.target_agent:
                    result.target_agent.append(target_agent)
    return result


def reset_event(start_time):
    return Event(start_time=start_time, duration=0, target_agent=None, action_type='idle')

In [ ]:
from pydantic import BaseModel
class Message(BaseModel):
    """
    Message class for communication between backend and frontend.
    """
    content: str
    agent_id: int
    action: str

    @classmethod
    def from_dict(cls, message_dict):
        return cls(
            message_dict["agent_id"], message_dict["action"], message_dict["content"]
        )

class RecommenderStat(BaseModel):
    tot_user_num: int
    cur_user_num: int
    tot_item_num: int
    inter_num: int
    rec_model: str
    pop_items: list[str]

class SocialStat(BaseModel):
    tot_user_num: int
    cur_user_num: int
    tot_link_num: int
    chat_num: int
    cur_chat_num: int
    post_num: int
    pop_items: list[str]
    network_density: float

In [ ]:
from datetime import datetime, timedelta
import calendar


class Interval:
    """
    Interval class for each round.
    """
    def __init__(self, value, unit):
        self.value = value
        self.unit = unit


def parse_interval(string):
    value = int(string[:-1])
    unit = string[-1]

    if unit == "s":
        return Interval(value, "seconds")
    elif unit == "m":
        return Interval(value, "minutes")
    elif unit == "h":
        return Interval(value, "hours")
    elif unit == "d":
        return Interval(value, "days")
    elif unit == "M":
        return Interval(value, "months")
    elif unit == "y":
        return Interval(value, "years")
    else:
        raise ValueError("Invalid unit: {}".format(unit))


def add_interval(date, interval):
    if interval.unit == "seconds":
        return date + timedelta(seconds=interval.value)
    elif interval.unit == "minutes":
        return date + timedelta(minutes=interval.value)
    elif interval.unit == "hours":
        return date + timedelta(hours=interval.value)
    elif interval.unit == "days":
        return date + timedelta(days=interval.value)
    elif interval.unit == "months":
        # Adding months to a date is a bit more complex
        new_year = date.year + (date.month + interval.value - 1) // 12
        new_month = (date.month + interval.value - 1) % 12 + 1
        new_day = min(date.day, calendar.monthrange(new_year, new_month)[1])
        return date.replace(year=new_year, month=new_month, day=new_day)
    elif interval.unit == "years":
        # Adding years to a date is similar to adding months
        new_year = date.year + interval.value
        new_month = date.month
        new_day = min(date.day, calendar.monthrange(new_year, new_month)[1])
        return date.replace(year=new_year, month=new_month, day=new_day)
    else:
        raise ValueError("Invalid interval unit: {}".format(interval.unit))


def format_time(date):
    return date.strftime("%B %d, %Y, %I:%M %p")


def get_weekday(date):
    return date.strftime("%A")

In [ ]:
import csv
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os
import math
import numpy as np


class Data:
    """
    Data class for loading data from local files.
    """

    def __init__(self, config):
        self.config = config
        self.items = {}
        self.users = {}
        self.db = None
        self.tot_relationship_num = 0
        self.netwerk_density = 0.0
        self.role_id = -1
        self.interrating = {}
        self.user_ratings = {}
        self.item_ratings = {}
        self.load_items(config["item_path"])
        self.load_users(config["user_path"])
        self.load_relationship(config["relationship_path"])
        self.load_faiss_db(config["index_name"])
        self.load_interactions_rating(config["rating_path"])

    def load_faiss_db(self, index_name):
        """
        Load faiss db from local if exists, otherwise create a new one.

        """
        embeddings = OpenAIEmbeddings()
        try:
            self.db = FAISS.load_local(index_name, embeddings)
            print("Load faiss db from local")
        except:
            titles = [item["name"] for item in self.items.values()]
            self.db = FAISS.from_texts(titles, embeddings)
            self.db.save_local(index_name)

    def load_relationship(self, file_path):
        """
        Load relationship of agents from local file.
        """
        with open(file_path, "r", newline="") as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                user_1, user_2, relationship, _ = row
                user_1 = int(user_1)
                user_2 = int(user_2)

                if user_1 not in self.users or user_2 not in self.users:
                    continue
                if "contact" not in self.users[user_1]:
                    self.users[user_1]["contact"] = dict()

                self.users[user_1]["contact"][user_2] = relationship
                self.tot_relationship_num += 1

    def load_items(self, file_path):
        """
        Load items from local file.
        """
        with open(file_path, "r", newline="") as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header line
            for row in reader:
                item_id, title, genre, description = row
                self.items[int(item_id)] = {
                    "name": title.strip(),
                    "genre": genre,
                    "description": description.strip(),
                    "inter_cnt": 0,
                    "mention_cnt": 0,
                }

    def load_users(self, file_path):
        """
        Load users from local file.
        """
        cnt = 1
        with open(file_path, "r", newline="") as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header line
            for row in reader:
                # print(len(row), row)
                user_id, name, gender, age, traits, status, interest, feature = row

                self.users[cnt] = {
                    "name": name,
                    "gender": gender,
                    "age": int(age),
                    "traits": traits,
                    "status": status,
                    "interest": interest,
                    "feature": feature,
                }
                cnt += 1

    def load_role(
        self, id, name, gender, age, traits, status, interest, feature, relationships
    ):
        """
        @ Zeyu Zhang
        Load the role user into this `Data` object. Then other agents can interact with the role.
        """
        self.role_id = id
        self.users[id] = {
            "name": name,
            "gender": gender,
            "age": int(age),
            "traits": traits,
            "status": status,
            "interest": interest,
            "feature": feature,
        }
        for id2, rel_value in relationships.items():
            if "contact" not in self.users[id]:
                self.users[id]["contact"] = dict()
            self.users[id]["contact"][id2] = rel_value

    def load_interactions_rating(self, file_path):
      """
      Load user-item interactions (with rating) from local file.
      Stores in self.interrating as a dict:
      {user_id: [(item_id, rating), ...], ...}
      """
      with open(file_path, "r", newline="") as file:
        reader = csv.reader(file)
        header = next(reader)  # Skip the header line
        for row in reader:
            user_id, item_id, rating = row
            user_id = int(user_id)
            item_id = int(item_id)
            rating = int(rating)
            if user_id not in self.interrating:
                self.interrating[user_id] = []
            self.interrating[user_id].append((item_id, rating))

            if user_id not in self.user_ratings:
                self.user_ratings[user_id] = []
            self.user_ratings[user_id].append(rating)

            # Store item rating
            if item_id not in self.item_ratings:
                self.item_ratings[item_id] = []
            self.item_ratings[item_id].append(rating)

      # Compute and store average historical rating for each user
      self.user_avg_rating = {uid: sum(ratings)/len(ratings) for uid, ratings in self.user_ratings.items() if ratings}

      # Compute and store average historical rating for each item
      self.item_avg_rating = {iid: sum(ratings)/len(ratings) for iid, ratings in self.item_ratings.items() if ratings}


    def get_full_items(self):
        return list(self.items.keys())

    def get_inter_popular_items(self):
        """
        Get the most popular items based on the number of interactions.
        """
        ids = sorted(
            self.items.keys(), key=lambda x: self.items[x]["inter_cnt"], reverse=True
        )[:3]
        return self.get_item_names(ids)

    def add_inter_cnt(self, item_names):
        item_ids = self.get_item_ids(item_names)
        print("item ids:", item_ids)
        for item_id in item_ids:
            self.items[item_id]["inter_cnt"] += 1

    def add_mention_cnt(self, item_names):
        item_ids = self.get_item_ids(item_names)
        for item_id in item_ids:
            self.items[item_id]["mention_cnt"] += 1

    def get_mention_popular_items(self):
        """
        Get the most popular items based on the number of mentions.
        """
        ids = sorted(
            self.items.keys(), key=lambda x: self.items[x]["mention_cnt"], reverse=True
        )[:3]
        return self.get_item_names(ids)

    def get_item_names(self, item_ids):
        return ["<" + self.items[item_id]["name"] + ">" for item_id in item_ids]

    def get_item_ids(self, item_names):
        item_ids = []
        for item in item_names:
            for item_id, item_info in self.items.items():
                if item_info["name"] in item:
                    item_ids.append(item_id)
                    break
        return item_ids

    def get_item_ids_exact(self, item_names):
        """
        Get item ids from item names.
        I coundn't find any difference with the get_item_ids(item_names) function
        """
        item_ids = []
        for item in item_names:
            for item_id, item_info in self.items.items():
                if item_info["name"] == item:
                    item_ids.append(item_id)
                    break
        return item_ids

    def get_full_users(self):
        return list(self.users.keys())

    def get_user_profile(self, user_id):
        """
        Return the user profile as a formatted string for the given user_id.
        """
        user = self.users.get(user_id)
        if not user:
           return f"User ID {user_id} not found."

        profile = (
        f"User ID: {user_id};; Name: {user['name']}\n"
        f"Gender: {user['gender']};; Age: {user['age']};; Status: {user['status']}\n"
        f"Traits: {user['traits']};; Interest: {user['interest']};; Feature: {user['feature']}\n")
        return profile

    def get_user_names(self, user_ids):
        return [self.users[user_id]["name"] for user_id in user_ids]

    def get_user_ids(self, user_names):
        user_ids = []
        for user in user_names:
            for user_id, user_info in self.users.items():
                if user_info["name"] == user:
                    user_ids.append(user_id)
                    break
        return user_ids

    def get_user_num(self):
        """
        Return the number of users.
        """
        return len(self.users.keys())

    def get_item_num(self):
        """
        Return the number of items.
        """
        return len(self.items.keys())

    def get_relationship_num(self):
        """
        Return the number of relationships.
        """
        return self.tot_relationship_num

    def get_role_id(self):
        """
        Return the number of relationships.
        """
        return self.role_id

    def search_items(self, item, k=50):
        """
        Search similar items from faiss db.
        Args:
            item: str, item name
            k: int, number of similar items to return
        """
        docs = self.db.similarity_search(item, k)
        item_names = [doc.page_content for doc in docs]
        return item_names

    def get_all_contacts(self, user_id):
        """
        Get all contacts of a user.
        """
        if "contact" not in self.users[user_id]:
            print(f"{user_id} has no contact.")
            return []
        ids = []
        for id in self.users[user_id]["contact"]:
            if id < self.get_user_num():
                ids.append(id)
        return self.get_user_names(ids)

    def get_all_contacts_id(self, user_id):
        """
        Get all contacts of a user.
        """
        if "contact" not in self.users[user_id]:
            print(f"{user_id} has no contact.")
            return []
        ids = []
        for id in self.users[user_id]["contact"]:
            if id < self.get_user_num():
                ids.append(id)
        return ids

    def get_relationships(self, user_id):
        """
        Get all relationships of a user.
        """
        if "contact" not in self.users[user_id]:
            print(f"{user_id} has no contact.")
            return dict()
        return self.users[user_id]["contact"]

    def get_relationship_names(self, user_id):
        """
        Get all relationship IDs of a user.
        """
        if "contact" not in self.users[user_id]:
            print(f"{user_id} has no contact.")
            return dict()
        relatiobnships = dict()
        for id in self.users[user_id]["contact"]:
            relatiobnships[self.users[id]["name"]] = self.users[user_id]["contact"][id]
        return relatiobnships

    def get_network_density(self):
        """
        Get the network density of the social network.
        """
        self.network_density = round(
            self.tot_relationship_num
            * 2
            / (self.get_user_num() * (self.get_user_num() - 1)),
            2,
        )
        return self.network_density

    def get_item_description_by_id(self, item_ids):
        """
        Get description of items by item id.
        """
        return [self.items[item_id]["description"] for item_id in item_ids]

    def get_item_description_by_name(self, item_names):
        """
        Get description of items by item name.
        """
        item_descriptions = []
        for item in item_names:
            found = False
            for item_id, item_info in self.items.items():
                if item_info["name"] == item.strip(" <>"):
                    item_descriptions.append(item_info["description"])
                    found = True
                    break
            if not found:
                item_descriptions.append("")
        return item_descriptions

    def get_genres_by_id(self, item_ids):
        """
        Get genre of items by item id.
        """
        # return [self.items[item_id]["genre"] for item_id in item_ids]
        return [
            genre
            for item_id in item_ids
            for genre in self.items[item_id]["genre"].split('|')
        ]

    def hit_at_k(self, ground_truth, predicted, k):
        """Return 1 if any of the top-k predicted are relevant, else 0."""
        return int(bool(set(ground_truth) & set(predicted[:k])))

    def ndcg_at_k(self, ground_truth, predicted, k):
        """Compute NDCG@k for a single user."""
        def dcg(rel):
          return np.sum([(2**r - 1) / np.log2(i + 2) for i, r in enumerate(rel)])

        rel = [1 if item in ground_truth else 0 for item in predicted[:k]]
        ideal_rel = sorted([1]*min(len(ground_truth), k) + [0]*(k - min(len(ground_truth), k)), reverse=True)
        dcg_score = dcg(rel)
        idcg_score = dcg(ideal_rel)
        return dcg_score / idcg_score if idcg_score > 0 else 0.0

    def mse(self, ground_truth_ratings, predicted_ratings, items=None):
        """Compute MSE for ratings (on items in both sets)."""
        if items is None:
           items = set(ground_truth_ratings.keys()) & set(predicted_ratings.keys())
        else:
           items = set(items) & set(ground_truth_ratings.keys()) & set(predicted_ratings.keys())
        if not items:
           return np.nan
        errors = [(ground_truth_ratings[i] - predicted_ratings[i]) ** 2 for i in items]
        return np.mean(errors)

    def rmse(self, ground_truth_ratings, predicted_ratings, items=None):
        """Compute RMSE for ratings."""
        return np.sqrt(self.mse(ground_truth_ratings, predicted_ratings, items))

    def safe_log(self, x):
        """Numerically safe log."""
        return math.log(max(x, 1e-15))

    def ordered_probit_probs(self, pred_int, K, taus=None):
        """
        Compute ordered probit class probabilities for a predicted integer rating.

        pred_int : int
            The predicted integer rating (e.g., 1..K).
        K : int
            Number of rating classes (e.g., 5 for 1–5 stars).
        taus : list or array, optional
            Thresholds separating the ordered categories.
            If None, uses equally spaced thresholds [1.5, 2.5, ..., K-0.5].
        """

        if taus is None:
           taus = np.array([1.5 + i for i in range(K-1)])  # default thresholds

        assert len(taus) == K-1

        def Phi(z):
            return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))  # Normal CDF

        probs = []
        for k in range(1, K+1):
           if k == 1:
              lower = -np.inf
              upper = taus[0]
           elif k == K:
              lower = taus[-1]
              upper = np.inf
           else:
              lower = taus[k-2]
              upper = taus[k-1]
           p_lower = 0.0 if lower == -np.inf else Phi((lower - pred_int))
           p_upper = 1.0 if upper == np.inf else Phi((upper - pred_int))
           probs.append(max(p_upper - p_lower, 1e-15))

        probs = np.array(probs)
        probs /= probs.sum()  # normalize
        return probs

In [ ]:
class BaseModel(object):
    """Base class for all models."""

    def __init__(self, config, n_users, n_items):
        self.config = config
        self.items = None

    def get_full_sort_items(self, user_id, *args, **kwargs):
        """Get a list of sorted items for a given user."""
        raise NotImplementedError

    def _sort_full_items(self, user_id, *args, **kwargs):
        """Sort a list of items for a given user."""
        raise NotImplementedError

In [ ]:
from typing import Union, List, Optional
import torch
import torch.nn as nn


class MF(BaseModel, nn.Module):
    def __init__(self, config, n_users, n_items):
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)
        self.config = config
        self.embedding_size = config["embedding_size"]
        self.n_users = n_users
        self.n_items = n_items
        torch.manual_seed(config['seed'])
        # define layers and loss
        self.user_embedding = nn.Embedding(self.n_users+1, self.embedding_size)
        self.item_embedding = nn.Embedding(self.n_items+1, self.embedding_size)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, user, item):
        """Predicts the rating of a user for an item."""
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)

        # Dot product between user and item embeddings to predict rating
        predicted_rating = (user_embed * item_embed).sum(1)

        return predicted_rating

    def get_full_sort_items(self, user, items):
        """Get a list of sorted items for a given user."""
        predicted_ratings = self.forward(user, items)
        sorted_items = self._sort_full_items(user, predicted_ratings, items)
        return sorted_items.tolist()

    def _sort_full_items(self, user, predicted_ratings, items):
        """Sort items based on their predicted ratings."""
        # Sort items based on ratings in descending order and return item indices
        _, sorted_indices = torch.sort(predicted_ratings, descending=True)
        return items[sorted_indices]

In [ ]:
import math
from typing import List, Tuple, Iterable, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class LightGCN(BaseModel, nn.Module):
    """
    LightGCN for recommendation.
    Reference: He et al., "LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation"
    """

    def __init__(
        self,
        config,
        n_users: int,
        n_items: int,
        interactions: Iterable[Tuple[int, int]],
        n_layers: int = 3,
        embedding_dim: int = 64,
        device: Optional[torch.device] = None,
    ):
        """
        interactions: iterable of (user_id, item_id) pairs (duplicates okay, handled as multi-edges of weight 1)
        """
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)

        self.n_users = n_users
        self.n_items = n_items
        self.n_layers = n_layers
        self.embedding_dim = embedding_dim
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        torch.manual_seed(config.get("seed", 2023))

        # Embeddings
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

        # Build normalized adjacency once
        self.Graph = self._build_normalized_adj(interactions).to(self.device)

        self.to(self.device)

    # ---------- Graph utilities ----------
    def _build_normalized_adj(self, interactions: Iterable[Tuple[int, int]]) -> torch.sparse.FloatTensor:
        """
        Build the symmetrically normalized adjacency matrix:
            A_hat = D^{-1/2} * A * D^{-1/2}
        for the bipartite user-item graph where nodes = users + items
        Size: (n_nodes, n_nodes) with n_nodes = n_users + n_items
        """
        n_nodes = self.n_users + self.n_items

        # Collect COO edges (user <-> item, undirected)
        rows = []
        cols = []
        vals = []

        for u, i, *rest in interactions:
            # Allow (u,i) or (u,i,rating) — rating ignored here (implicit 1)
            if u < 0 or u >= self.n_users or i < 0 or i >= self.n_items:
                continue
            i_offset = self.n_users + i  # item node index in unified graph
            # u -> i
            rows.append(u); cols.append(i_offset); vals.append(1.0)
            # i -> u
            rows.append(i_offset); cols.append(u); vals.append(1.0)

        if len(rows) == 0:
            # Empty graph fallback (identity to avoid NaNs)
            indices = torch.arange(n_nodes, dtype=torch.long)
            indices = torch.stack([indices, indices], dim=0)
            values = torch.ones(n_nodes, dtype=torch.float32)
            return torch.sparse_coo_tensor(indices, values, (n_nodes, n_nodes)).coalesce()

        indices = torch.tensor([rows, cols], dtype=torch.long)
        values = torch.tensor(vals, dtype=torch.float32)
        A = torch.sparse_coo_tensor(indices, values, (n_nodes, n_nodes)).coalesce()

        # Degree vector d = sum of rows
        deg = torch.sparse.sum(A, dim=1).to_dense()  # (n_nodes,)
        # Avoid divide-by-zero
        deg = torch.clamp(deg, min=1e-12)
        d_inv_sqrt = torch.pow(deg, -0.5)

        # Normalize values: for each edge (i,j), val *= d^-1/2[i] * d^-1/2[j]
        row, col = A.indices()
        norm_vals = A.values() * d_inv_sqrt[row] * d_inv_sqrt[col]

        A_hat = torch.sparse_coo_tensor(A.indices(), norm_vals, A.size()).coalesce()
        return A_hat

    # ---------- Embedding propagation ----------
    def propagate(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Perform K-layer LightGCN propagation and return final user & item embeddings.
        E^(0) = concat([U, I])  -> shape (n_users + n_items, d)
        E_final = 1/(K+1) * sum_{k=0..K} E^(k)
        """
        E_u0 = self.user_embedding.weight
        E_i0 = self.item_embedding.weight
        E0 = torch.cat([E_u0, E_i0], dim=0)  # (n_users + n_items, d)

        all_layers = [E0]
        x = E0
        for _ in range(self.n_layers):
            x = torch.sparse.mm(self.Graph, x)  # LightGCN propagation (no weights, no nonlinearity)
            all_layers.append(x)

        E = torch.stack(all_layers, dim=0).mean(dim=0)  # layer-wise average
        Eu, Ei = torch.split(E, [self.n_users, self.n_items], dim=0)
        return Eu, Ei

    # ---------- Scoring ----------
    def forward(self, users: torch.Tensor, items: torch.Tensor) -> torch.Tensor:
        """
        Predict scores for (users, items). Supports:
          - users: (B,), items: (B,)  -> elementwise scores
          - users: (B,), items: (N,)  -> broadcast users for all N items (returns (B,N))
        """
        users = users.to(self.device)
        items = items.to(self.device)

        Eu, Ei = self.propagate()

        if users.dim() == 1 and items.dim() == 1 and users.shape[0] == items.shape[0]:
            u_emb = Eu[users]                   # (B, d)
            i_emb = Ei[items]                   # (B, d)
            return (u_emb * i_emb).sum(dim=1)   # (B,)

        # Broadcast to (B, N)
        u_emb = Eu[users]                       # (B, d)
        i_emb = Ei[items]                       # (N, d)
        scores = u_emb @ i_emb.t()              # (B, N)
        return scores

    def predict(self, user_ids: List[int], item_ids: Optional[List[int]] = None) -> np.ndarray:
        """
        Convenience method to get scores as numpy.
        - If item_ids is None: scores for all items for each user -> shape (B, n_items)
        - Else: scores for user_ids x item_ids -> (B, len(item_ids))
        """
        self.eval()
        with torch.no_grad():
            u = torch.tensor(user_ids, dtype=torch.long, device=self.device)
            if item_ids is None:
                items = torch.arange(self.n_items, dtype=torch.long, device=self.device)
                scores = self.forward(u, items)        # (B, n_items)
            else:
                items = torch.tensor(item_ids, dtype=torch.long, device=self.device)
                scores = self.forward(u, items)        # (B, len(item_ids))
        return scores.detach().cpu().numpy()

    # ---------- Ranking API (BaseModel) ----------
    def get_full_sort_items(self, user_id: int, seen_items: Optional[Iterable[int]] = None, top_k: Optional[int] = None) -> List[int]:
        """
        Rank all items for a given user (descending by predicted score).
        Optionally drop previously seen items.
        """
        scores = self.predict([user_id], None).ravel()  # (n_items,)
        if seen_items is not None:
            # push seen items to bottom
            seen_items = [i for i in seen_items if 0 <= i < self.n_items]
            scores[np.array(seen_items, dtype=np.int64)] = -np.inf

        order = np.argsort(-scores)  # descending
        if top_k is not None:
            order = order[:top_k]
        return order.tolist()

    def _sort_full_items(self, user_id: int, predicted_ratings: torch.Tensor, items: torch.Tensor):
        # Not used here; keeping for BaseModel compatibility
        _, idx = torch.sort(predicted_ratings, descending=True)
        return items[idx]


class BPRLoss(nn.Module):
    """
    Pairwise Bayesian Personalized Ranking loss with L2 regularization on embeddings.
    """
    def __init__(self, reg: float = 1e-4):
        super().__init__()
        self.reg = reg

    def forward(self, u_emb, pos_emb, neg_emb):
        pos_scores = (u_emb * pos_emb).sum(dim=1)
        neg_scores = (u_emb * neg_emb).sum(dim=1)
        loss = -F.logsigmoid(pos_scores - neg_scores).mean()
        reg = (u_emb.norm(2).pow(2) + pos_emb.norm(2).pow(2) + neg_emb.norm(2).pow(2)) / u_emb.shape[0]
        return loss + self.reg * reg

In [ ]:
import os
from typing import Optional, Tuple, Iterable, List, Dict
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


class InteractionDataset(Dataset):
    """
    Dataset that yields dense user vectors (torch.float32).
    Accepts a scipy.sparse CSR matrix or a numpy array of shape (n_users, n_items).
    """
    def __init__(self, user_item_matrix):
        # Accept CSR matrix or numpy array
        if hasattr(user_item_matrix, "toarray") and hasattr(user_item_matrix, "tocsr"):
            self.mat = user_item_matrix.tocsr()
            self.is_sparse = True
            self.n_users, self.n_items = self.mat.shape
        else:
            self.mat = np.asarray(user_item_matrix, dtype=np.float32)
            self.is_sparse = False
            self.n_users, self.n_items = self.mat.shape

    def __len__(self):
        return self.n_users

    def __getitem__(self, idx):
        if self.is_sparse:
            row = self.mat.getrow(idx).toarray().astype(np.float32).squeeze(0)
            return torch.from_numpy(row)
        else:
            return torch.from_numpy(self.mat[idx].astype(np.float32))


class MultiVAE(BaseModel, nn.Module):
    """
    MultiVAE model (variational autoencoder for collaborative filtering).
    - encoder: two-layer MLP -> mu, logvar
    - decoder: linear mapping from latent z to item logits
    """
    def __init__(
        self,
        config: dict,
        n_users: int,
        n_items: int,
        hidden_dim: int = 600,
        latent_dim: int = 200,
        dropout=0.5,
        device: Optional[torch.device] = None,
    ):
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)

        self.device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        self.n_users = n_users
        self.n_items = n_items

        # model dims
        self.hidden_dim = config.get("vae_hidden_dim", hidden_dim)
        self.latent_dim = config.get("vae_latent_dim", latent_dim)
        self.dropout = config.get("vae_dropout", dropout)

        # encoder
        self.encoder_fc1 = nn.Linear(n_items, self.hidden_dim)
        self.encoder_fc2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.mu_layer = nn.Linear(self.hidden_dim, self.latent_dim)
        self.logvar_layer = nn.Linear(self.hidden_dim, self.latent_dim)

        # decoder (linear + bias to produce logits over items)
        self.decoder = nn.Linear(self.latent_dim, n_items, bias=True)

        self.act = nn.Tanh()
        self.drop = nn.Dropout(self.dropout)

        # initialization
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

        self.to(self.device)

    # ---------- VAE ops ----------
    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        x: (B, n_items) input user vectors
        returns mu, logvar (each (B, latent_dim))
        """
        h = self.drop(self.act(self.encoder_fc1(x)))
        h = self.drop(self.act(self.encoder_fc2(h)))
        mu = self.mu_layer(h)
        logvar = self.logvar_layer(h)
        return mu, logvar

    def reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """
        Returns logits for items: (B, n_items)
        """
        logits = self.decoder(z)
        return logits

    def forward(self, x: torch.Tensor, sample: bool = True) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        One forward pass:
        - encodes x -> mu, logvar
        - draws z (reparameterization)
        - decodes logits
        Returns: logits, mu, logvar
        """
        mu, logvar = self.encode(x)
        if self.training and sample:
            z = self.reparameterize(mu, logvar)
        else:
            z = mu  # use mean for deterministic inference
        logits = self.decode(z)
        return logits, mu, logvar

    # ---------- Loss / ELBO ----------
    def loss_function(
        self,
        logits: torch.Tensor,
        input_batch: torch.Tensor,
        mu: torch.Tensor,
        logvar: torch.Tensor,
        anneal: float = 1.0,
    ) -> Tuple[torch.Tensor, float, float]:
        """
        Multinomial likelihood version used in MultiVAE:
        recon_loss = - sum_j x_j * log_softmax(logits)_j

        KL = -0.5 * sum(1 + logvar - mu^2 - exp(logvar))
        total_loss = recon_loss + anneal * KL
        Returns (loss, recon_loss_scalar, kl_scalar)
        """
        # reconstruction: log softmax and weighted by counts
        log_softmax = F.log_softmax(logits, dim=1)  # (B, n_items)
        # input_batch may be counts (0/1 or counts). multiply and sum per sample
        recon = -torch.sum(log_softmax * input_batch, dim=1)  # (B,)
        recon_loss = torch.mean(recon)

        # KL
        kl = -0.5 * torch.sum(1.0 + logvar - mu.pow(2) - logvar.exp(), dim=1)  # (B,)
        kl_loss = torch.mean(kl)

        loss = recon_loss + anneal * kl_loss
        return loss, recon_loss.item(), kl_loss.item()


    # ---------- Embeddings helper (for ranking similar to LightGCN propagate) ----------
    def get_user_latent(self, x: torch.Tensor) -> torch.Tensor:
        """
        Return deterministic latent mu for x (no sampling).
        x: (B, n_items)
        returns mu: (B, latent_dim)
        """
        mu, logvar = self.encode(x)
        return mu

    # ---------- Prediction / Ranking (BaseModel methods) ----------
    def predict(self, user_vectors: torch.Tensor) -> np.ndarray:
        """
        Predict item scores for a batch of user vectors.
        user_vectors: torch.Tensor (B, n_items) on cpu or device
        returns: numpy array (B, n_items) of logits (higher = more recommended)
        """
        self.eval()
        with torch.no_grad():
            user_vectors = user_vectors.to(self.device)
            logits, mu, logvar = self.forward(user_vectors, sample=False)
            # Return logits (no softmax) so they can be sorted. move to cpu
            return logits.detach().cpu().numpy()

    def get_full_sort_items(self, user_id: int, user_item_matrix, seen_items: Optional[Iterable[int]] = None, top_k: Optional[int] = None) -> List[int]:
        """
        Rank all items for a given user.
        - user_item_matrix: the full user-item matrix (csr or numpy) to extract the user's vector
        - seen_items: optional iterable to mask (push to bottom)
        """
        # build user vector
        if hasattr(user_item_matrix, "getrow"):
            vec = user_item_matrix.getrow(user_id).toarray().astype(np.float32)
            user_vec = torch.from_numpy(vec).to(self.device)
        else:
            user_vec = torch.from_numpy(np.asarray(user_item_matrix[user_id], dtype=np.float32)).to(self.device)

        scores = self.predict(user_vec.unsqueeze(0)).ravel()  # (n_items,)

        if seen_items is not None:
            seen = [i for i in seen_items if 0 <= i < self.n_items]
            scores[np.array(seen, dtype=np.int64)] = -np.inf

        order = np.argsort(-scores)
        if top_k is not None:
            order = order[:top_k]
        return order.tolist()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader



class FMModel(BaseModel, nn.Module):
    def __init__(self, config, n_users, n_items, n_factors=16):
        BaseModel.__init__(self, config, n_users, n_items)
        nn.Module.__init__(self)
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors

        # Linear terms
        self.user_bias = nn.Embedding(n_users, 1)
        self.item_bias = nn.Embedding(n_items, 1)

        # Factorization embeddings
        self.user_embedding = nn.Embedding(n_users, n_factors)
        self.item_embedding = nn.Embedding(n_items, n_factors)

        # Global bias
        self.global_bias = nn.Parameter(torch.zeros(1))

        # Init
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.item_bias.weight)

    def forward(self, user_ids, item_ids):
        user_vec = self.user_embedding(user_ids)
        item_vec = self.item_embedding(item_ids)

        linear_terms = self.user_bias(user_ids) + self.item_bias(item_ids)
        interaction = torch.sum(user_vec * item_vec, dim=1, keepdim=True)

        score = self.global_bias + linear_terms + interaction
        return score.view(-1)

    def get_full_sort_items(self, user_id):
        device = next(self.parameters()).device
        user_id_tensor = torch.tensor([user_id], device=device)
        item_ids = torch.arange(self.n_items, device=device)
        user_id_expand = user_id_tensor.expand(self.n_items)
        scores = self.forward(user_id_expand, item_ids)
        return torch.argsort(scores, descending=True).tolist()

    def _sort_full_items(self, user_id):
        return self.get_full_sort_items(user_id)

In [ ]:
from __future__ import annotations
import math
import random
from typing import List, Dict, Tuple, Iterable

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict


# Utility: set all seeds
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def sasrec_pointwise_step(model, batch, device, logit_clip=20.0):
    users, seqs, pos_items, neg_items, mask = batch
    seqs, pos_items, neg_items = seqs.to(device), pos_items.to(device), neg_items.to(device)
    mask = mask.to(device).bool()

    seq_out = model(seqs)
    item_emb = model.item_embedding.weight

    pos_logits = torch.sum(seq_out * item_emb[pos_items], dim=-1).clamp(-logit_clip, logit_clip)
    neg_logits = torch.sum(seq_out * item_emb[neg_items], dim=-1).clamp(-logit_clip, logit_clip)

    valid_mask = (pos_items > 0) & mask
    if valid_mask.sum() == 0:
        return torch.tensor(0.0, device=device, requires_grad=True)

    loss_pos = F.binary_cross_entropy_with_logits(pos_logits[valid_mask], torch.ones_like(pos_logits[valid_mask]))
    loss_neg = F.binary_cross_entropy_with_logits(neg_logits[valid_mask], torch.zeros_like(neg_logits[valid_mask]))

    loss = loss_pos + loss_neg

    # L2 reg, skip padding embedding
    if model.l2_emb > 0:
        loss += model.l2_emb * (item_emb[1:].norm(p=2) ** 2) / 2

    return loss


# Dataset & Collate
class SASRecDataset(Dataset):
    """Builds sequences for SASRec training.

    Args:
        user2items: dict mapping user_id -> list of interacted item ids (in time order)
        n_items: total number of items (max ID)
        max_seq_len: truncate/pad sequences to this length
        min_seq_len: smallest effective length (>=2 to have a next item)
    """
    def __init__(self,
                 user2items: Dict[int, List[int]],
                 n_items: int,
                 max_seq_len: int = 50,
                 min_seq_len: int = 2):
        self.user2items = user2items
        self.users = [u for u, seq in user2items.items() if len(seq) >= min_seq_len]
        self.n_items = n_items
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx: int):
        user = self.users[idx]
        full = self.user2items[user]
        # Truncate to latest max_seq_len
        seq = full[-self.max_seq_len:]
        return user, seq


def _pad_sequence(seq: list, max_len: int) -> list:
    """Left-pad sequence with 0 to max_len."""
    seq = seq[-max_len:]
    return [0] * (max_len - len(seq)) + seq

def _build_pos_items(seq: list) -> list:
    """Next-item targets, last position padded with 0."""
    return seq[1:] + [0]


def _sample_negatives(seq: list, n_items: int) -> list:
    """Sample negatives for each position, avoiding seq items."""
    user_set = set(seq)
    negatives = []
    for _ in range(len(seq)):
        neg = random.randint(1, n_items - 1)  # avoid 0
        while neg in user_set:
            neg = random.randint(1, n_items - 1)
        negatives.append(neg)
    return negatives


def sasrec_collate(batch, n_items: int, max_seq_len: int):
    users, seqs = zip(*batch)
    seqs = [_pad_sequence(s, max_seq_len) for s in seqs]

    pos_items = [_build_pos_items(s) for s in seqs]
    neg_items = [_sample_negatives(s, n_items) for s in seqs]
    mask = [[1 if x != 0 else 0 for x in s] for s in seqs]

    return (
        torch.tensor(users, dtype=torch.long),
        torch.tensor(seqs, dtype=torch.long),
        torch.tensor(pos_items, dtype=torch.long),
        torch.tensor(neg_items, dtype=torch.long),
        torch.tensor(mask, dtype=torch.float),
    )


def build_user2items(train_data):
    user2items = defaultdict(list)
    for u, i, r in sorted(train_data, key=lambda x: (x[0], x[2])):
        user2items[u].append(i)
    return user2items


# ------------------------------
# Model
# ------------------------------
class SASRec(nn.Module, BaseModel):
    def __init__(self, config, n_users: int, n_items: int):
        nn.Module.__init__(self)  # initialize nn.Module
        BaseModel.__init__(self, config, n_users, n_items)  # keep BaseModel logic

        self.n_items = n_items
        self.hidden_units = int(config.get("hidden_units", 128))
        self.max_seq_len = int(config.get("max_seq_len", 50))
        self.num_heads = int(config.get("num_heads", 2))
        self.num_blocks = int(config.get("num_blocks", 2))
        self.dropout_rate = float(config.get("dropout_rate", 0.2))
        self.l2_emb = float(config.get("l2_emb", 0.0))

        # Embeddings
        self.item_embedding = nn.Embedding(n_items + 1, self.hidden_units, padding_idx=0)
        self.position_embedding = nn.Embedding(self.max_seq_len, self.hidden_units)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=self.hidden_units,
                nhead=self.num_heads,
                dim_feedforward=self.hidden_units * 4,
                dropout=self.dropout_rate,
                activation="gelu",
                batch_first=True,
            ) for _ in range(self.num_blocks)
        ])
        self.dropout = nn.Dropout(self.dropout_rate)
        self.layer_norm = nn.LayerNorm(self.hidden_units)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.item_embedding.weight, std=0.02)
        nn.init.normal_(self.position_embedding.weight, std=0.02)

    def _causal_mask(self, L: int, device=None):
        # True means masked in PyTorch
        mask = torch.triu(torch.ones(L, L, device=device), diagonal=1).bool()
        return mask

    def forward(self, item_seq: torch.Tensor) -> torch.Tensor:
        """
        Encode sequence safely, with padding & causal masks.
        Args:
            item_seq: (B, L) padded with 0 on left
        Returns:
            seq_out: (B, L, H)
        """
        B, L = item_seq.shape
        device = item_seq.device
        pos_ids = torch.arange(L, device=device).unsqueeze(0).expand(B, L)

        # Embeddings
        x = self.item_embedding(item_seq) + self.position_embedding(pos_ids)
        x = self.layer_norm(x)
        x = self.dropout(x)

        key_padding_mask = (item_seq == 0)  # True = pad
        attn_mask = self._causal_mask(L, device=device)  # True = masked

        for blk in self.blocks:
            x = blk(
            x,
            src_mask=attn_mask,
            src_key_padding_mask=key_padding_mask
            )
            # Safety: replace NaN/inf
            if torch.isnan(x).any() or torch.isinf(x).any():
               x = torch.nan_to_num(x, nan=0.0, posinf=1e4, neginf=-1e4)

        return x

    @torch.no_grad()
    def get_full_sort_items(self, user_id, user_seq: torch.Tensor):
        """Return sorted item IDs by score for a single user.
        Args:
            user_seq: (1, L)
        Returns:
            torch.Tensor of sorted item ids (desc)
        """
        self.eval()
        seq_out = self.forward(user_seq)[:, -1, :]  # (1, H)
        all_item_emb = self.item_embedding.weight  # (n_items+1, H)
        scores = torch.matmul(seq_out, all_item_emb.t()).squeeze(0)  # (n_items+1,)
        # Avoid recommending padding id 0
        scores[0] = -1e9
        return torch.argsort(scores, descending=True)

    def _sort_full_items(self, user_id, *args, **kwargs):
        raise NotImplementedError("Use get_full_sort_items(user_id, user_seq)")

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import random
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
from torch.utils.data import DataLoader


class Recommender:
    """
    Recommender System class
    """

    def __init__(self, config, logger, data):
        self.data = data
        self.config = config
        self.logger = logger
        self.page_size = config["page_size"]
        self.items_per_page = config["items_per_page"]
        self.random_k = config["rec_random_k"]
        self.train_data = []
        self.n_layers = 3
        self.embedding_dim = 4
        if config["rec_model"] == "MF":
           self.model = MF(config, self.data.get_user_num(), self.data.get_item_num())
        elif config["rec_model"] == "LightGCN":
           self.model = LightGCN(config, self.data.get_user_num(), self.data.get_item_num(), self.train_data, n_layers=self.n_layers, embedding_dim=self.embedding_dim)
        elif config["rec_model"] == "SASRec":
           self.model = SASRec(config, self.data.get_user_num(), self.data.get_item_num())
        elif config["rec_model"] == "MultiVAE":
           self.model = MultiVAE(config, self.data.get_user_num(), self.data.get_item_num())
        elif config["rec_model"] == "FM":
           self.model = FMModel(config, self.data.get_user_num(), self.data.get_item_num())
        else:
           raise ValueError(f"Unknown model: {config['rec_model']}")

        self.criterion = nn.MSELoss()
        if config["rec_model"] != "FM":
           self.optimizer = optim.Adam(self.model.parameters(), lr=config["lr"], weight_decay=1e-5)

        self.epoch_num = config["epoch_num"]
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.record = {}
        self.round_record = {}
        self.positive = {}
        self.interaction_dict = {}
        self.inter_df = None
        self.inter_num = 0
        for user in self.data.get_full_users():
            self.record[user] = []
            self.positive[user] = []
            self.round_record[user] = []
        self.user_data = {
            "user": [],
            "N_expose": [],
            "N_view": [],
            "N_like": [],
            "N_exit": [],
            "S_sat": []
            }
        self.rating_feeling = {
            "User": [],
            "Rating": [],
            "Feelings": []
        }

    def sample_bpr_triples(self,
                           user_pos_items: List[List[int]],
                           n_items: int,
                           batch_size: int,
                           device: torch.device,
                           ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        user_pos_items: list of lists; for each user u, a list of items they've interacted with (positive)
        returns (users, pos_items, neg_items) tensors of length batch_size
        """
        users = []
        pos = []
        neg = []
        for _ in range(batch_size):
           # sample a user with at least one positive
           while True:
               u = np.random.randint(0, len(user_pos_items))
               if len(user_pos_items[u]) > 0:
                  break

           i = np.random.choice(user_pos_items[u])
           # sample a negative item

           while True:
               j = np.random.randint(0, n_items)
               if j not in user_pos_items[u]:
                  break

           users.append(u)
           pos.append(i)
           neg.append(j)

        return (
        torch.tensor(users, dtype=torch.long, device=self.device),
        torch.tensor(pos, dtype=torch.long, device=self.device),
        torch.tensor(neg, dtype=torch.long, device=self.device),
        )

    def train_lightgcn_bpr(self,
        reg: float = 1e-4,
        log: bool = True):
        """
        Train LightGCN with BPR loss.
        - train_interactions/val_interactions can be (u,i) or (u,i,rating>0) tuples.
        - If ckpt_path is provided, saves the best (by simple val recall proxy) state dict.
        """

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config["lr"])
        self.criterion = BPRLoss(reg=reg)

        # Build user -> positives list
        user_pos = [[] for _ in range(self.model.n_users)]
        for u, i, *rest in self.train_data:
            if 0 <= u < self.model.n_users and 0 <= i < self.model.n_items:
               user_pos[u].append(i)

        # Basic validation proxy: count of positives ranked in top-10 (cheap & optional)
        def quick_val_topk_hits(k: int = 10) -> float:
            if self.val_data is None:
               return -1.0
            # build val positives per user
            val_pos = [[] for _ in range(self.model.n_users)]
            for u, i, *rest in self.val_data:
               if 0 <= u < self.model.n_users and 0 <= i < self.model.n_items:
                   val_pos[u].append(i)

            hits = 0
            total = 0

            for u in range(self.model.n_users):
               if not val_pos[u]:
                  continue
               recs = self.model.get_full_sort_items(u, seen_items=set(user_pos[u]), top_k=k)
               s = set(val_pos[u])
               hits += len([r for r in recs if r in s])
               total += min(k, len(s))
            return hits / total if total > 0 else -1.0

        best_metric = -math.inf

        # Build full checkpoint file path
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_lightGCN_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        for epoch in range(1, self.epoch_num + 1):
            self.model.train()

            # One epoch of mini-batch BPR
            n_steps = max(1, sum(len(v) for v in user_pos) // max(1, config['batch_size']))
            losses = []
            for _ in range(n_steps):
                users, pos_items, neg_items = self.sample_bpr_triples(user_pos, self.model.n_items, config['batch_size'], self.device)
                Eu, Ei = self.model.propagate()
                u_emb = Eu[users]
                p_emb = Ei[pos_items]
                n_emb = Ei[neg_items]
                loss = self.criterion(u_emb, p_emb, n_emb)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                losses.append(loss.item())

            # quick validation metric
            metric = quick_val_topk_hits(k=10)
            if log:
               print(f"[Epoch {epoch:3d}] BPR Loss: {np.mean(losses):.4f} | Val@10: {metric:.4f}")

            # Save checkpoint if validation improves
            if metric > best_metric:
               best_metric = metric
               torch.save({
                   "epoch": self.epoch_num + 1,
                   "model_state_dict": self.model.state_dict(),
                   "n_users": self.data.get_user_num(),
                   "n_items": self.data.get_item_num(),
                   "n_layers": self.n_layers,
                   "embedding_dim": self.embedding_dim,
                   "metric": metric,
                   }, ckpt_file)
               self.logger.info(f"Best model updated at epoch {epoch+1}, saved to {ckpt_file}")

        # Load best (optional)
        if ckpt_file is not None and best_metric > -math.inf:
           # At the end, reload the best weights for inference
           checkpoint = torch.load(ckpt_file)
           self.model.load_state_dict(checkpoint['model_state_dict'])

    def bce_sampled_loss(self, seq_out: torch.Tensor,
        pos_items: torch.Tensor,
        neg_items: torch.Tensor,
        item_embedding: nn.Embedding,
        mask: torch.Tensor,
        l2_emb: float = 0.0) -> torch.Tensor:
        """Binary cross-entropy on sampled positives/negatives per position.
        Args:
            seq_out: (B, L, H)
            pos_items: (B, L) next-item ids (0 where no target)
            neg_items: (B, L) sampled negatives (0 where no target)
            item_embedding: embedding module (to fetch item vectors)
            mask: (B, L) boolean, True where a target exists (i.e., pos_items > 0)
            l2_emb: weight decay on item embeddings (regularizes pos/neg lookups)
        """
        B, L, H = seq_out.shape

        pos_vecs = item_embedding(pos_items)  # (B, L, H)
        neg_vecs = item_embedding(neg_items)  # (B, L, H)

        pos_logits = (seq_out * pos_vecs).sum(-1)  # (B, L)
        neg_logits = (seq_out * neg_vecs).sum(-1)  # (B, L)

        # Targets: pos -> 1, neg -> 0
        pos_loss = F.binary_cross_entropy_with_logits(pos_logits[mask], torch.ones_like(pos_logits[mask]))
        neg_loss = F.binary_cross_entropy_with_logits(neg_logits[mask], torch.zeros_like(neg_logits[mask]))
        loss = pos_loss + neg_loss

        if l2_emb > 0:
           reg = (pos_vecs[mask].pow(2).sum() + neg_vecs[mask].pow(2).sum()) / mask.sum().clamp_min(1)
           loss = loss + l2_emb * reg
        return loss

    def train_sasrec(self, grad_clip=1.0, logit_clip=20.0):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Build user->items dict and dataset
        user2items = build_user2items(self.train_data)
        n_items_global = int(self.data.get_item_num())
        max_seq_len = self.model.max_seq_len

        train_dataset = SASRecDataset(user2items, n_items=n_items_global, max_seq_len=max_seq_len)
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.config['batch_size'],
            shuffle=True,
            collate_fn=lambda batch: sasrec_collate(batch, n_items=n_items_global, max_seq_len=max_seq_len)
            )

        # Validation loader
        val_user2items = build_user2items(self.val_data)
        val_dataset = SASRecDataset(val_user2items, n_items=n_items_global, max_seq_len=max_seq_len)
        val_loader = DataLoader(
            val_dataset,
            batch_size=self.config['batch_size'],
            shuffle=False,
            collate_fn=lambda batch: sasrec_collate(batch, n_items=n_items_global, max_seq_len=max_seq_len)
            )

        # Checkpoint setup
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_SASRec_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)
        best_metric = -float("inf")

        for epoch in range(1, self.epoch_num + 1):
           self.model.train()
           running = 0.0
           n_steps = 0

           for users, seqs, pos, neg, umask in train_loader:
              users, seqs, pos, neg, umask = (
                users.to(self.device),
                seqs.to(self.device),
                pos.to(self.device),
                neg.to(self.device),
                umask.to(self.device).bool(),  # ensure bool type
                )

              # Compute mask dynamically
              mask = pos > 0
              mask = mask.bool()
              if mask.sum() == 0:
                 continue  # skip batch with no valid positions

              # Compute stable loss
              loss = sasrec_pointwise_step(self.model, (users, seqs, pos, neg, mask), device=self.device, logit_clip=logit_clip)

              self.optimizer.zero_grad(set_to_none=True)
              loss.backward()
              if grad_clip is not None:
                 nn.utils.clip_grad_norm_(self.model.parameters(), grad_clip)
              self.optimizer.step()

              running += loss.item()
              n_steps += 1

           avg_loss = running / max(1, n_steps)
           print(f"Epoch {epoch}/{self.epoch_num} - train loss: {avg_loss:.4f}")

           # Validation
           if val_loader is not None:
              self.model.eval()
              val_loss = 0.0
              n_val_steps = 0
              with torch.no_grad():
                  for users, seqs, pos, neg, umask in val_loader:
                    #  print("Users:", users)
                    #  print("Pos min/max:", pos.min().item(), pos.max().item())
                    #  print("Neg min/max:", neg.min().item(), neg.max().item())
                     users, seqs, pos, neg, umask = (
                         users.to(self.device),
                         seqs.to(self.device),
                         pos.to(self.device),
                         neg.to(self.device),
                         umask.to(self.device).bool(),)

                     mask = pos > 0
                     if mask.sum() == 0:
                        # print("Skipped empty batch")
                        continue

                     loss = sasrec_pointwise_step(self.model, (users, seqs, pos, neg, mask), device=self.device, logit_clip=logit_clip)
                    #  print("Batch loss:", loss.item())
                     val_loss += loss.item()
                     n_val_steps += 1
              val_loss /= max(1, n_val_steps)
              metric = -val_loss
              print(f"Epoch {epoch}, Val Loss: {val_loss:.4f}")
           else:
              metric = -avg_loss

           # Save best model
           if metric > best_metric:
              best_metric = metric
              torch.save({
                "epoch": epoch,
                "model_state_dict": self.model.state_dict(),
                "metric": metric,
              }, ckpt_file)
              print(f"Best model updated at epoch {epoch}, saved to {ckpt_file}")

        # Load best model after training
        checkpoint = torch.load(ckpt_file, map_location=self.device)
        self.model.load_state_dict(checkpoint["model_state_dict"])


    def train_fm(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.config['lr'])
        self.criterion = nn.MSELoss()

        best_metric = -math.inf
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_FM_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        # 🔹 Wrap datasets into DataLoaders
        train_loader = DataLoader(self.train_data, batch_size=self.config['batch_size'], shuffle=True)
        val_loader = None
        if self.val_data is not None:
           val_loader = DataLoader(self.val_data, batch_size=self.config['batch_size'], shuffle=False)


        for epoch in range(self.epoch_num):
           self.model.train()
           total_loss = 0

           for user, item, rating in train_loader:  # must be DataLoader
              user, item, rating = user.to(self.device), item.to(self.device), rating.float().to(self.device)

              self.optimizer.zero_grad()
              preds = self.model(user, item)
              loss = self.criterion(preds, rating)
              loss.backward()
              self.optimizer.step()

              total_loss += loss.item()

           avg_loss = total_loss / len(train_loader)
           print(f"Epoch {epoch+1}/{self.epoch_num}, Train Loss: {avg_loss:.4f}")

           metric = None
           if val_loader is not None:
              self.model.eval()
              with torch.no_grad():
                  val_loss = 0
                  for user, item, rating in val_loader:
                      user, item, rating = user.to(self.device), item.to(self.device), rating.to(self.device)
                      preds = self.model(user, item)
                      val_loss += self.criterion(preds, rating).item()
                  val_loss /= len(val_loader)

              metric = -val_loss
              print(f"Epoch {epoch+1}, Val Loss: {val_loss:.4f}")
           else:
              metric = -avg_loss  # fallback if no validation set

           # save best model
           if metric > best_metric:
              best_metric = metric
              torch.save({
                "epoch": epoch+1,
                "model_state_dict": self.model.state_dict(),
                "n_users": self.model.n_users,
                "n_items": self.model.n_items,
                "n_factors": self.model.n_factors,
                "metric": metric,
                }, ckpt_file)
              print(f"Best model updated at epoch {epoch+1}, saved to {ckpt_file}")

        checkpoint = torch.load(ckpt_file, map_location=self.device)
        self.model.load_state_dict(checkpoint["model_state_dict"])


    def train_multivae(self,
        weight_decay: float = 0.0,
        anneal_cap: float = 0.2,
        total_anneal_steps: int = 200000,
        patience: int = 100,
        verbose: bool = True,):
        """
        Train MultiVAE on user-item matrix (CSR or dense numpy).
        - anneal_cap: maximum beta for KL weighting
        - total_anneal_steps: number of optimization steps over which to ramp beta from 0->anneal_cap
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        dataset = InteractionDataset(self.train_matrix)
        loader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True, drop_last=False)

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config['lr'], weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.5)
        update_count = 0
        best_val_loss = float("inf")

        # Build full checkpoint file path
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MultiVAE_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        wait = 0

        # AMP scaler (for mixed precision training)
        scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

        # validation dataset (create once, not per epoch)
        val_loader, val_dataset = None, None
        if self.val_matrix is not None:
           val_dataset = InteractionDataset(self.val_matrix)
           val_loader = DataLoader(val_dataset, batch_size=self.config['batch_size'], shuffle=False)

        for epoch in range(1, self.epoch_num + 1):
            self.model.train()
            epoch_loss, epoch_recon, epoch_kl, n_batches = 0.0, 0.0, 0.0, 0

            for batch in loader:
                batch = batch.to(self.device).float()
                assert batch.shape[1] == self.model.n_items, "Batch dimension mismatch!"

                # anneal factor
                if total_anneal_steps > 0:
                   anneal = min(anneal_cap, update_count / total_anneal_steps)
                else:
                   anneal = anneal_cap

                self.optimizer.zero_grad()

                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    logits, mu, logvar = self.model(batch, sample=True)
                    # clamp logvar inside model (optional, numerical stability)
                    logvar = torch.clamp(logvar, min=-10, max=10)
                    loss, recon_l, kl_l = self.model.loss_function(logits, batch, mu, logvar, anneal)

                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=5.0)
                scaler.step(self.optimizer)
                scaler.update()

                epoch_loss += loss.item()
                epoch_recon += recon_l
                epoch_kl += kl_l
                update_count += 1
                n_batches += 1

            scheduler.step()
            avg_train_elbo = epoch_loss / n_batches

            if verbose:
               print(f"[Epoch {epoch}] Train ELBO: {avg_train_elbo:.4f} | "
                  f"Recon: {epoch_recon / n_batches:.4f} | KL: {epoch_kl / n_batches:.4f} | "
                  f"Anneal: {anneal:.4f}")

            # ---------- Validation ----------
            if val_loader is not None:
               self.model.eval()
               val_losses = []
               with torch.no_grad():
                   for vb in val_loader:
                      vb = vb.to(self.device).float()
                      logits, mu, logvar = self.model(vb, sample=False)
                      logvar = torch.clamp(logvar, min=-10, max=10)
                      vloss, vrec, vkl = self.model.loss_function(logits, vb, mu, logvar, anneal)
                      val_losses.append(vloss.item() * len(vb))

               val_loss = float(np.sum(val_losses) / len(val_dataset))

               if verbose:
                 print(f"  -> Val ELBO: {val_loss:.4f}")

               # save best with early stopping
               if val_loss < best_val_loss:
                  best_val_loss = val_loss
                  wait = 0
                  torch.save({
                    "epoch": epoch,
                    "model_state_dict": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict(),
                    "n_users": self.data.get_user_num(),
                    "n_items": self.data.get_item_num(),
                    "hidden_dim": self.config["vae_hidden_dim"],
                    "latent_dim": self.config["vae_latent_dim"],
                    "config": self.config,
                    }, ckpt_file)
                  self.logger.info(f"Best model updated at epoch {epoch}, saved to {ckpt_file}")
               else:
                  wait += 1
                  if wait >= patience:
                     print(f"Early stopping triggered at epoch {epoch}")
                     break

        # reload best model
        checkpoint = torch.load(ckpt_file, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

    def load_model(self, path):
        self.model.load_state_dict(torch.load(path))

    def swap_items(self, lst, page_size, random_k):
        total_pages = len(lst) // page_size
        lst = lst[: total_pages * page_size]
        for page in range(1, total_pages // 2 + 1):
            start_idx = (page - 1) * page_size
            end_idx = start_idx + page_size - 1
            symmetric_start_idx = (total_pages - page) * page_size
            symmetric_end_idx = symmetric_start_idx + page_size

            for k in range(1, random_k + 1):
                lst[end_idx - k], lst[symmetric_end_idx - k] = (
                    lst[symmetric_end_idx - k],
                    lst[end_idx - k],
                )

        return lst

    def add_random_items(self, user, item_ids):
        item_ids = self.swap_items(item_ids, self.page_size, self.random_k)
        return item_ids

    def ordered_probit_loglik(self, y_true, y_pred_int, K=5, taus=None):
        """
        Compute log-likelihood for ordered probit model given integer predictions.

        y_true : list or array
           True ratings (1..K).
        y_pred_int : list or array
           Predicted integer ratings (1..K).
        K : int
           Number of rating categories (default 5).
        taus : list or array, optional
           Thresholds (default: equally spaced).
        """

        assert len(y_true) == len(y_pred_int), "Mismatch in true vs predicted length"
        ll = 0.0
        for t, p in zip(y_true, y_pred_int):
           probs = self.data.ordered_probit_probs(p, K, taus)
           ll += self.data.safe_log(probs[t-1])  # subtract 1 for 0-based index
        avg_ll = ll / len(y_true)
        return ll, avg_ll

    def update_user_interactions(self, user_id, new_items):
        """
        Updates the directory of user_id and interacted_items.
        - interaction_dict: dict mapping user_id -> set of interacted item ids
        - user_id: int or str
        - new_items: iterable of item ids (list, set, etc)

        After calling, interaction_dict[user_id] contains all unique interacted items.
        """
        # Ensure the user's interaction set exists
        if user_id not in self.interaction_dict:
          self.interaction_dict[user_id] = set()

        new_items = set(new_items) - self.interaction_dict[user_id]
        self.interaction_dict[user_id].update(new_items)

    def get_full_manual_items(self, user_id, gt_ratio, rd_ratio, total_items, read = None, heard = None):
        """
        Get a list of manual items for a given user.
        """
        gtruth_items = self.data.interrating[user_id]
        gt_items = [item for item, rating in gtruth_items]
        rd_items = self.data.get_full_items()

        # Remove any gt_items from rd_items to avoid duplicates
        rd_items = list(set(rd_items) - set(gt_items))

        # 1. Determine counts
        total_ratio = gt_ratio + rd_ratio
        gt_count = round(total_items * gt_ratio / total_ratio)
        rd_count = total_items - gt_count

        # Make sure we don't try to sample more than available
        gt_count = min(gt_count, len(gt_items))
        rd_count = min(rd_count, len(rd_items))

        # 2. Randomly sample
        chosen_gt = random.sample(gt_items, gt_count) if gt_count > 0 else []
        chosen_rd = random.sample(rd_items, rd_count) if rd_count > 0 else []

        # 3. Combine and shuffle if desired
        final_items = chosen_gt + chosen_rd

        # items discriptions
        sorted_item_names = self.data.get_item_names(final_items)
        description = self.data.get_item_description_by_id(final_items)
        eb_item = [
            sorted_item_names[i]
            + ";;"
            + description[i]
            + ";; Genre: "
            + self.data.get_genres_by_id([final_items[i]])[0]
            for i in range(len(sorted_item_names))
        ]
        return final_items, eb_item, chosen_gt

    def get_full_sort_items(self, user, random=False):
        """
        Get a list of sorted items for a given user.
        """
        items = self.data.get_full_items()
        user_tensor = torch.tensor(user)
        items_tensor = torch.tensor(items)
        sorted_items = self.model.get_full_sort_items(user_tensor, items_tensor)
        if self.random_k > 0 and random == True:
            sorted_items = self.add_random_items(user, sorted_items)
        sorted_items = [item for item in sorted_items if item not in self.record[user]]
        sorted_item_names = self.data.get_item_names(sorted_items)
        description = self.data.get_item_description_by_id(sorted_items)
        items = [
            sorted_item_names[i]
            + ";;"
            + description[i]
            + ";; Genre: "
            + self.data.get_genres_by_id([sorted_items[i]])[0]
            for i in range(len(sorted_item_names))
        ]
        return sorted_items, items

    def get_item(self, idx):
        item_name = self.data.get_item_names([idx])[0]
        description = self.data.get_item_description_by_id([idx])[0]
        item = item_name + ";;" + description
        return item

    def get_search_items(self, item_name):
        return self.data.search_items(item_name)

    def get_inter_num(self):
        return self.inter_num

    def update_history_by_name(self, user_id, item_names):
        """
        Update the history of a given user.
        """
        item_names = [item_name.strip(" <>'\"") for item_name in item_names]
        item_ids = self.data.get_item_ids(item_names)
        self.record[user_id].extend(item_ids)

    def update_history_by_id(self, user_id, item_ids):
        """
        Update the history of a given user.
        """
        self.record[user_id].extend(item_ids)

    def update_positive(self, user_id, item_names):
        """
        Update the positive history of a given user.
        """
        item_ids = self.data.get_item_ids(item_names)
        if len(item_ids) == 0:
            return
        self.positive[user_id].extend(item_ids)
        self.inter_num += len(item_ids)

    def update_positive_by_id(self, user_id, item_id):
        """
        Update the history of a given user.
        """
        self.positive[user_id].append(item_id)

    def save_interaction(self):
        """
        Save the interaction history to a csv file.
        """
        inters = []
        users = self.data.get_full_users()
        for user in users:
            for item in self.positive[user]:
                new_row = {"user_id": user, "item_id": item, "rating": 1}
                inters.append(new_row)

            for item in self.record[user]:
                if item in self.positive[user]:
                    continue
                new_row = {"user_id": user, "item_id": item, "rating": 0}
                inters.append(new_row)

        df = pd.DataFrame(inters)
        df.to_csv(
            self.config["interaction_path"],
            index=False,
        )

        self.inter_df = df

    def add_train_data(self, user, item, label):
        self.train_data.append((user, item, label))

    def clear_train_data(self):
        self.train_data = []

    def add_user(self, user_id, N_expose, N_view, N_like, N_exit, S_sat):
        self.user_data["user"].append(user_id)
        self.user_data["N_expose"].append(N_expose)
        self.user_data["N_view"].append(N_view)
        self.user_data["N_like"].append(N_like)
        self.user_data["N_exit"].append(N_exit)
        self.user_data["S_sat"].append(S_sat)

    def add_review(self, user_id, rating, feelings):
        self.rating_feeling["User"].append(user_id)
        self.rating_feeling["Rating"].append(rating)
        self.rating_feeling["Feelings"].append(feelings)

    def satisfaction_metrics(self):
        sm_df = pd.DataFrame(self.user_data)
        if len(sm_df) == 0:
           return None  # no data yet

        metrics = {}
        sm_df["view_ratio"] = sm_df["N_view"] / sm_df["N_expose"]
        sm_df["like_ratio"] = sm_df["N_like"] / sm_df["N_expose"]

        metrics["P_view"] = sm_df["view_ratio"].mean()
        metrics["N_like"] = sm_df["N_like"].mean()
        metrics["P_like"] = sm_df["like_ratio"].mean()
        metrics["N_exit"] = sm_df["N_exit"].mean()
        metrics["S_sat"] = sm_df["S_sat"].mean()

        return metrics

    def get_entropy(
        self,
    ):
        tot_entropy = 0
        for user in self.record.keys():
            inters = self.record[user]
            genres = self.data.get_genres_by_id(inters)
            entropy = calculate_entropy(genres)
            tot_entropy += entropy

        return tot_entropy / len(self.record.keys())

    def check_train_data(self):
        """
        Print or inspect the training data.
        """
        print("Training Data:")
        for user, item, label in self.train_data:
            print(f"User: {user}, Item: {item}, Label: {label}")

    def create_train_data(self):
        """
        Create a training dataset with random samples.

        Args:
            num_samples (int): Number of samples to generate.
        """
        self.clear_train_data()  # Clear existing training data
        all_data = self.data.interrating  # You need to implement this or use available interaction data

        # Convert dict to list of (user, item, label)
        # triplets = []
        for user, interactions in all_data.items():
            for item, rating in interactions:
                self.add_train_data(user, item, float(rating))
                # triplets.append((user, item, float(rating)))  # keep exact rating

        # Split 80% train, 20% temp (to further split into val/test)
        self.train_data, self.temp_data = train_test_split(self.train_data, test_size=0.2, random_state=2025)

        # Split temp into 10% val and 10% test (from the total dataset)
        self.val_data, self.test_data = train_test_split(self.temp_data, test_size=0.5, random_state=2025)

        train_users = max([u for u, i, r in self.train_data]) + 1
        train_items = max([i for u, i, r in self.train_data]) + 1

        val_users = max([u for u, i, r in self.val_data]) + 1
        val_items = max([i for u, i, r in self.val_data]) + 1

        test_users = max([u for u, i, r in self.test_data]) + 1
        test_items = max([i for u, i, r in self.test_data]) + 1

        n_items_global = int(self.data.get_item_num())

        # Initialize user-item matrix
        self.train_matrix = np.zeros((train_users, n_items_global), dtype=np.float32)
        self.val_matrix = np.zeros((val_users, n_items_global), dtype=np.float32)
        self.test_matrix = np.zeros((test_users, n_items_global), dtype=np.float32)

        # Fill interactions safely
        for u, i, r in self.train_data:
           if i >= n_items_global: continue  # skip bad indices
           self.train_matrix[u, i] = r

        for u, i, r in self.val_data:
           if i >= n_items_global: continue
           self.val_matrix[u, i] = r

        for u, i, r in self.test_data:
           if i >= n_items_global: continue
           self.test_matrix[u, i] = r



    def calculate_user_metrics(
        self, user_id, sim_recommended, all_items, threshold = 3):
        """
        Evaluate precision, recall, (optionally real) accuracy, and F1 for a single user.

        Returns:
            dict: { 'precision': float, 'recall': float, 'accuracy': float, 'f1': float }
        """

        if user_id not in self.data.interrating:
           return {'precision': 0, 'recall': 0, 'accuracy': 0, 'f1': 0}

        ground_truth_pairs = self.data.interrating[user_id]
        gt_relevant = set(item for item, rating in ground_truth_pairs if rating >= threshold and item in all_items)
        sim_recommended = set(sim_recommended)
        all_items = set(all_items)

        TP = len(gt_relevant & sim_recommended)
        FP = len(sim_recommended - gt_relevant)
        FN = len(gt_relevant - sim_recommended)
        TN = len(all_items - (gt_relevant | sim_recommended))

        precision = TP / (TP + FP) if (TP + FP) else 0.0
        recall = TP / (TP + FN) if (TP + FN) else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0.0
        accuracy = (TP + TN) / len(all_items) if all_items else 0

        print("precision:", precision, "recall:", recall, "accuracy:", accuracy, "f1:", f1)
        return precision, recall, accuracy, f1

    def precisionandrecallk(
        self, user_id, recommended, k):
        if user_id not in self.data.interrating:
           return {'precision_at_k': 0, 'recall_at_k': 0}

        sim_recommended = list(dict.fromkeys(recommended))
        ground_truth_pairs = self.data.interrating[user_id]
        gt_relevant = set(item for item, rating in ground_truth_pairs if rating >= 3)
        recommended_at_k = sim_recommended[:k]
        hits = sum([1 for item in recommended_at_k if item in gt_relevant])
        precision_at_k = hits / k
        recall_at_k = hits / len(gt_relevant) if gt_relevant else 0
        return precision_at_k, recall_at_k


    def calculation_of_rating(self, user_id, item_names, book_rating):
        item_ids = self.data.get_item_ids([item_names])
        if user_id in self.data.interrating:
           # Check for item in user's ratings
           for (itm, rating) in self.data.interrating[user_id]:
               if itm == item_ids[0]:
                  return (rating, book_rating)

        # If not found
        return (0, book_rating)


    def calc_mse_rmse_rating_percentages(self, rating_pairs):

        print("Incoming rating_pairs:", rating_pairs[:20])  # show first 20 pairs
        print("Total pairs:", len(rating_pairs))

        # Remove pairs with zero in ground truth or predicted rating
        filtered_pairs = [(gt, pred) for gt, pred in rating_pairs
                          if int(gt) != 0]

        print("After filtering:", filtered_pairs[:20])
        print("Remaining pairs:", len(filtered_pairs))

        if not filtered_pairs:
           # No valid data after filtering
           return None, None, {}, {}, None, None, None

        # Convert ratings to int
        gt = [int(gt) for gt, pred in filtered_pairs]
        pred = [int(pred) for gt, pred in filtered_pairs]
        mse = np.mean([(g - p) ** 2 for g, p in zip(gt, pred)])
        rmse = np.sqrt(mse)
        loglike, ob_loglike = self.ordered_probit_loglik(gt, pred)
        rho, p_value = spearmanr(gt, pred)

        gt_count = Counter(gt)
        pred_count = Counter(pred)
        total = len(filtered_pairs)

        gt_pct = {r: gt_count.get(r, 0) / total * 100 for r in range(1, 6)}
        pred_pct = {r: pred_count.get(r, 0) / total * 100 for r in range(1, 6)}
        return mse, rmse, gt_pct, pred_pct, loglike, ob_loglike, rho


    def test_recommendations(self, user_id):
        # Get the full list of items
        all_items = self.data.get_full_items()

        # Convert the user ID to tensor
        user_tensor = torch.tensor(user_id)

        # Convert all items to tensor
        items_tensor = torch.tensor(all_items)

        # Get sorted items based on the model's prediction
        sorted_items = self.model.get_full_sort_items(user_tensor, items_tensor)

        # Filter out items that are already in the user's history
        recommended_items = [item for item in sorted_items if item not in self.record[user_id]]

        # Return the recommended items
        return recommended_items

    def evaluate(self, dataset):
        self.model.eval()
        users = torch.tensor([x[0] for x in dataset])
        items = torch.tensor([x[1] for x in dataset])
        labels = torch.tensor([x[2] for x in dataset]).float()

        with torch.no_grad():
             outputs = self.model(users, items)
             loss = self.criterion(outputs, labels)
        return loss.item()

    def load_checkpoint(self, path="best_model.pth", resume_training=False):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        if resume_training:
           # Load optimizer state to resume training exactly where it left off
           self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
           start_epoch = checkpoint["epoch"]
           self.logger.info(f"Resuming training from epoch {start_epoch}")
           return start_epoch
        else:
           self.model.eval()  # set to eval mode for inference

    def train_mf(self):
        if len(self.train_data) == 0:
            print("No training data!")
            return

        users = [x[0] for x in self.train_data]
        items = [x[1] for x in self.train_data]
        labels = [x[2] for x in self.train_data]


        dataset = torch.utils.data.TensorDataset(
        torch.tensor(users), torch.tensor(items), torch.tensor(labels))

        train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=self.config["batch_size"], shuffle=True)

        self.model.train()

        best_val_loss = float("inf")

        # Build full checkpoint file path
        ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MF_model.pth")
        os.makedirs(self.config['checkpoint_path'], exist_ok=True)

        for epoch in range(self.epoch_num):
            epoch_loss = 0.0

            for user, item, label in train_loader:

                self.optimizer.zero_grad()
                outputs = self.model(user, item)
                loss = self.criterion(outputs, label.float())
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()

            val_loss = self.evaluate(self.val_data)  # Evaluate on validation set

            self.logger.info(
            f"Epoch {epoch+1}/{self.epoch_num}, Train Loss: {epoch_loss/len(train_loader):.4f}, "
            f"Val Loss: {val_loss:.4f}")

            # Save checkpoint if validation improves
            if val_loss < best_val_loss:
               best_val_loss = val_loss
               torch.save({
                "epoch": epoch + 1,
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "val_loss": val_loss,
                }, ckpt_file)
               self.logger.info(f"Best model updated at epoch {epoch+1}, saved to {ckpt_file}")

        # At the end, reload the best weights for inference
        checkpoint = torch.load(ckpt_file)
        self.model.load_state_dict(checkpoint['model_state_dict'])

    def load_best_model(self):
        if self.config['rec_model'] == 'MF':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MF_model.pth")
        elif self.config['rec_model'] == 'MultiVAE':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_MultiVAE_model.pth")
        elif self.config['rec_model'] == 'LightGCN':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_lightGCN_model.pth")
        elif self.config['rec_model'] == 'FM':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_FM_model.pth")
        elif self.config['rec_model'] == 'SASRec':
           ckpt_file = os.path.join(self.config['checkpoint_path'], "best_SASRec_model.pth")
        else:
           raise ValueError(f"Unknown model type: {self.config['rec_model']}")

        # Build full checkpoint file path
        checkpoint = torch.load(ckpt_file)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        self.logger.info(f"Loaded best model from {ckpt_file}")

    def get_rec_discription(self, final_items):
        # items discriptions
        sorted_item_names = self.data.get_item_names(final_items)
        description = self.data.get_item_description_by_id(final_items)
        eb_item = [
            sorted_item_names[i]
            + ";;"
            + description[i]
            + ";; Genre: "
            + self.data.get_genres_by_id([final_items[i]])[0]
            for i in range(len(sorted_item_names))
        ]
        return eb_item

    def get_full_rankings(self, use_test=False, batch_size=512):
        """
        Compute full rankings for all users in self.data.
        - training items are pushed to the end
        - optionally, ground-truth test items can be put on top
        """
        if self.config['rec_model'] == 'MF':
           n_users = self.data.get_user_num()
           n_items = self.data.get_item_num()

           item_embed = self.model.item_embedding.weight[:n_items, :]

           self.full_rankings = np.zeros((n_users, n_items), dtype=int)

           for user in range(n_users):
              user_tensor = torch.tensor([user])
              user_embed = self.model.user_embedding(user_tensor)

              scores = torch.matmul(user_embed, item_embed.T).squeeze(0).detach().numpy()

              # Only consider valid item indices
              train_items = [x[1] for x in self.train_data if x[0] == user and x[1] < n_items]
              scores[train_items] = -np.inf

              self.full_rankings[user] = np.argsort(-scores)

              # # Optionally move ground-truth test items on top
              if use_test:
                 test_items = [x[1] for x in self.test_data if x[0] == user and x[1] < n_items]
                 for idx, item in enumerate(test_items):
                    if item in self.full_rankings[user]:
                       current_pos = np.where(self.full_rankings[user] == item)[0][0]
                       self.full_rankings[user][idx], self.full_rankings[user][current_pos] = (
                         self.full_rankings[user][current_pos],
                         self.full_rankings[user][idx])

        elif self.config['rec_model'] == 'LightGCN':
            n_users = self.data.get_user_num()
            n_items = self.data.get_item_num()

            # === 1. Get all user/item embeddings from LightGCN ===
            self.model.eval()
            with torch.no_grad():
                all_user_emb, all_item_emb = self.model.propagate()
                # shapes: (n_users, embed_dim), (n_items, embed_dim)

            self.full_rankings = np.zeros((n_users, n_items), dtype=int)

            for user in range(n_users):
               # Get user embedding
               user_embed = all_user_emb[user].unsqueeze(0)   # (1, embed_dim)

               # Compute scores for all items (dot product)
               scores = torch.matmul(user_embed, all_item_emb.T).squeeze(0).cpu().numpy()

               # Push training items to -inf
               train_items = [x[1] for x in self.train_data if x[0] == user and x[1] < n_items]
               scores[train_items] = -np.inf

               # Sort descending
               self.full_rankings[user] = np.argsort(-scores)

               # # Optionally move ground-truth test items on top
               if use_test:
                  test_items = [x[1] for x in self.test_data if x[0] == user and x[1] < n_items]
                  for idx, item in enumerate(test_items):
                     if item in self.full_rankings[user]:
                        current_pos = np.where(self.full_rankings[user] == item)[0][0]
                        self.full_rankings[user][idx], self.full_rankings[user][current_pos] = (
                           self.full_rankings[user][current_pos],
                           self.full_rankings[user][idx])

        elif self.config['rec_model'] == 'MultiVAE':
            # n_users = self.data.get_user_num()
            # n_items = self.data.get_item_num()
            if use_test:
               matrix = self.test_matrix
            else:
               matrix = self.train_matrix

            n_users, n_items = matrix.shape

            self.model.eval()
            self.full_rankings = np.zeros((n_users, n_items), dtype=int)

            with torch.no_grad():
                for start in range(0, n_users, batch_size):
                   end = min(start + batch_size, n_users)

                   # === 1. Build user input batch (interaction vectors) ===
                   batch_users = []
                   for u in range(start, end):
                      row = self.train_matrix[u]  # should return (n_items,) vector
                      batch_users.append(row)
                   batch_users = torch.tensor(batch_users, dtype=torch.float32).to(self.model.device)

                   # === 2. Forward pass through MultiVAE ===
                   logits, mu, logvar = self.model(batch_users)   # shape: (batch_size, n_items)
                   scores = logits.cpu().numpy()

                   # === 3. Postprocess each user in batch ===
                   for i, u in enumerate(range(start, end)):
                      user_scores = scores[i]

                      train_items = [x[1] for x in self.train_data if x[0] == u and x[1] < n_items]
                      user_scores[train_items] = -np.inf

                      # Sort items by descending score
                      self.full_rankings[u] = np.argsort(-user_scores)

                      # Optionally move ground-truth test items on top
                      if use_test:
                         test_items = [x[1] for x in self.test_data if x[0] == u and x[1] < n_items]
                         for idx, item in enumerate(test_items):
                            if item in self.full_rankings[u]:
                               current_pos = np.where(self.full_rankings[u] == item)[0][0]
                               self.full_rankings[u][idx], self.full_rankings[u][current_pos] = (
                                   self.full_rankings[u][current_pos],
                                   self.full_rankings[u][idx])

        elif self.config['rec_model'] == 'FM':
            n_users = self.data.get_user_num()
            n_items = self.data.get_item_num()

            self.full_rankings = np.zeros((n_users, n_items), dtype=int)

            self.model.eval()
            device = next(self.model.parameters()).device

            for user in range(n_users):
               # Generate all item IDs
               item_ids = torch.arange(n_items, device=device)
               user_ids = torch.full((n_items,), user, dtype=torch.long, device=device)

               # Compute scores using FM forward
               with torch.no_grad():
                  scores = self.model(user_ids, item_ids).cpu().numpy()

               # Push training items to the end
               train_items = [x[1] for x in self.train_data if x[0] == user and x[1] < n_items]
               scores[train_items] = -np.inf

               # Sort items by descending score
               ranking = np.argsort(-scores)

               # Optionally move ground-truth test items to the top
               if use_test:
                  test_items = [x[1] for x in self.test_data if x[0] == user and x[1] < n_items]
                  for idx, item in enumerate(test_items):
                     if item in ranking:
                        current_pos = np.where(ranking == item)[0][0]
                        ranking[idx], ranking[current_pos] = ranking[current_pos], ranking[idx]

               self.full_rankings[user] = ranking

        elif self.config['rec_model'] == 'SASRec':
            n_users = self.data.get_user_num()
            n_items = self.data.get_item_num()

            self.full_rankings = np.zeros((n_users, n_items), dtype=int)
            self.model.eval()
            device = next(self.model.parameters()).device

            with torch.no_grad():
               for start in range(0, n_users, batch_size):
                  end = min(start + batch_size, n_users)
                  batch_users = list(range(start, end))

                  # Build input sequences for batch
                  batch_seqs = []
                  for u in batch_users:
                     # Get user interaction sequence from train_data
                     user_items = [x[1] for x in self.train_data if x[0] == u]
                     padded_seq = _pad_sequence(user_items, self.model.max_seq_len)
                     batch_seqs.append(padded_seq)

                  batch_seqs = torch.tensor(batch_seqs, dtype=torch.long, device=device)

                  # Forward pass: get sequence embeddings
                  seq_out = self.model(batch_seqs)  # (B, L, H)
                  seq_out_last = seq_out[:, -1, :]  # use last position (B, H)

                  # All item embeddings
                  all_item_emb = self.model.item_embedding.weight[:n_items, :]  # (n_items, H)

                  # Compute scores
                  scores = torch.matmul(seq_out_last, all_item_emb.T)  # (B, n_items)
                  scores = scores.cpu().numpy()

                  # Mask training items
                  for i, u in enumerate(batch_users):
                     train_items = [x[1] for x in self.train_data if x[0] == u and x[1] < n_items]
                     scores[i, train_items] = -np.inf  # push train items to the end

                     ranking = np.argsort(-scores[i])  # full ranking by score (highest first)
                     if use_test:
                        test_items = [x[1] for x in self.test_data if x[0] == u and x[1] < n_items]
                        # Keep only test items that appear in ranking
                        test_items_in_ranking = [item for item in ranking if item in test_items]
                        # Take at most 5 test items
                        top_test_items = test_items_in_ranking[:5]
                        # Remaining items (exclude the ones we forced to the top)
                        other_items = [item for item in ranking if item not in top_test_items]

                        # New ranking: top test items first, then the rest in score order
                        ranking = np.array(top_test_items + other_items)
                     # Store final ranking
                     self.full_rankings[u] = ranking


In [ ]:
import logging
import random
import re
from datetime import datetime
from typing import Any, Dict, List, Optional

import numpy as np
# from random import random
from langchain.embeddings import OpenAIEmbeddings
from langchain import LLMChain
from langchain.prompts import PromptTemplate

logger = logging.getLogger(__name__)

class SensoryMemory():
    """
    Sensory memory is intended to receive the observations (that are ready to be stored as memories) from the environment,
    extract and summarize important elements by attention mechanism, and output them to short term memory.
    """

    def __init__(self, llm, buffer_size=1):
        """
        Initialize the sensory memory.
        :param llm: The LLM object passed from RecAgentMemory.
        :param buffer_size (default as 1): Maximum number of observations. When len(self.buffer) >= buffer_size,
            then dump them as a piece of short term memory.
        """

        self.llm = llm
        self.buffer_size = buffer_size

        # Important weight can be used to adjust the balance between 'importance' and 'recency'.
        self.importance_weight = 0.9

        # Store a batch of observations.
        self.buffer = []

    def clear(self):
        """
        Clear the short term memory.
        """
        self.buffer = []

    def _score_memory_importance(self, observation: str) -> float:
        """
        Obtain the importance score of this memory.
        :param observation: The text of the observation.
        :return: (float) The importance of this observation.
        """
        prompt = PromptTemplate.from_template(
            """
            Please give an importance score between 1 to 10 for the following observation. Higher score indicates the observation is more important. More rules that should be followed are
            \n(1) The observation that includes entering social media is not important. e.g., David Smith takes action by entering the world of social media.
            \n(2) The observation that describes chatting with someone but no specific book name is not important. e.g., David Smith observed that David Miller expressed interest in chatting about books.
            \n(3) The observation that includes 'chatting' is not important. e.g., David Smith observed that David Miller expressed interest in chatting about movies, indicating a shared passion for films.
            \n(4) The observation that includes 'enter the recommender system' is not important. e.g. David Smith enters the Recommender System to explore book recommendations based on his interests and preferences.
            \n(5) The observation that recommends or mentions specific books is important.
            \n(6) More informative indicates more important, especially when two people are chatting.
            Please respond with a single integer.
            \nObservation:{observation}
            \nRating:
            """
        )
        score = LLMChain(llm=self.llm, prompt=prompt).run(observation=observation).strip()
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(match.group(1)) / 10) * self.importance_weight
        else:
            return 0.0

    def dump_shortTerm_list(self):
        """
        Convert all the observations in buffer to a piece of short term memory, and clear the buffer.
        :return: List of tuple (score[float], stm[str])
        """

        def parse_res(text: str):
            """
            Parse the output of LLM.
            """
            return [text]

        # Construct a string which includes all the observations in the buffer.
        obs_str = "The observations are as following:\n"
        for ind, obs in enumerate(self.buffer):
            obs_str += "[%d] %s\n" % (ind, obs)

        # Construct the order for converting.
        order_str = "You should summarize the above observation(s) into one independent sentence." \
                    "If there is a person's name in the observation, use third person, otherwise use first person. " \
                    "Note that the sentence should pay more attention to the book interest and the reasons in the " \
                    "observations." \
                    "The summarization should not include the profile explicitly."

        # Construct the prompt for LLM.
        prompt = PromptTemplate.from_template(obs_str + order_str)
        result = LLMChain(llm=self.llm, prompt=prompt).run({})
        result = parse_res(result)
        # Give the short term memory an importance score.
        result = [(self._score_memory_importance(text), text) for text in result]
        # Remove the short term memory whose importance score is lower than a threshold.
        result = [text for text in result if text[0] > 0.62]

        # Clear the buffer.
        self.clear()

        if len(result) != 0:
            return result
        else:
            return None

    def add_ssm(self, obs):
        """
        This function is only called in the function RecAgentMemory.save_context(). It is used to transport observations to a piece of short term memory.
        For each time, it receives only one observation, and adds into buffer. If buffer is full, then converts them into a piece of term memory.
        :param obs: The observation that is ready to transport to short term memory.
        :return: (1)Buffer full: List of tuple (score[float], stm[str]). (2) Buffer not full: None.
        """
        # Add the observation into the buffer.
        self.buffer.append(obs)

        # If the buffer is full, then dump and return the short term list to RecAgentMemory.
        # If the buffer is not full, then directly return 'None'.
        if len(self.buffer) >= self.buffer_size:
            return self.dump_shortTerm_list()
        else:
            return None

class ShortTermMemory():
    """
    The short-term memory module is to temporally store the observations from sensory memory module,
    which can be enhanced by other observations or retrieved memories to enter the long-term memory module.
    """
    def __init__(self, llm):
        self.llm = llm
        """The core language model."""
        self.verbose: bool = False

        self.capacity: int = 10
        """The capacity of Short-term memory"""

        self.short_memories: List[str] = []
        """The list of short-term memories"""

        self.short_embeddings: List[List[float]] = []
        """The OpenAI embeddings of short-term memories"""

        self.memory_importance: List[float] = []
        """The importance score list of short-term memories"""

        self.enhance_cnt: List[int] = [0 for _ in range(self.capacity)]
        """The number of enhancement of short-term memories"""

        self.enhance_memories: List[List[str]] = [[] for _ in range(self.capacity)]
        """The enhance memory of each short-term memory"""

        self.enhance_threshold: int = 3
        """Summary the short-term memory with enhanced count larger or equal than the threshold"""


    @staticmethod
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r"\n", text.strip())
        lines = [line for line in lines if line.strip()]  # remove empty lines
        return [re.sub(r"^\s*\d+\.\s*", "", line).strip() for line in lines]

    def chain(self, prompt: PromptTemplate) -> LLMChain:
        return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)

    def get_short_term_insight(self, content: str):
        """
        Get insight of the short-term memory and other memories or observations that enhance that short-term memory.
        :param content: short-term memory and other memories or observations that enhance that short-term memory
        :return: (List[str]) The insight of the short-term memory.
        """
        prompt = PromptTemplate.from_template(
            "There are some memories separated by semicolons (;): {content}\n"
            + "Can you infer from the above memories the high-level insight for this person's character?"
            + "The insight needs to be significantly different from the content and structure of the original memories."
            + "Respond in one sentence."
            + "\n\nResults:"
        )
        result = self.chain(prompt).run(content=content).strip()
        return self._parse_list(result)

    def transfer_memories(self, observation):
        """
        Transfer all possible short-term memories to long-term memory.
        :param observation: the observation enters the short-term memory or the retrieved memory
        :return
            (List[str]) memory_content: the enhanced short-term memories
            (List[float]) memory_importance: the importance scores of the enhanced short-term memories
            (List[List[str]]) insight_content: the insight from the short-term memories
        """
        # if the observation is summarized, otherwise add it into short-term memory
        transfer_flag = False
        existing_memory = [True for _ in range(len(self.short_memories))]
        memory_content, memory_importance, insight_content = [], [], []
        for idx, memory in enumerate(self.short_memories):
            if self.enhance_cnt[idx] >= self.enhance_threshold or existing_memory[idx] is True:
                existing_memory[idx] = False
                transfer_flag = True
                # combine all existing related memories to current memory in short-term memories
                content = [memory]
                # do not repeatedly add observation memory to summary, so use [:-1].
                for enhance_memory in self.enhance_memories[idx][:-1]:
                    content.append(enhance_memory)
                content.append(observation)
                content = ';'.join(content)
                memory_content.append(memory)
                # memory_importance.append(self.memory_importance[idx])
                insight = self.get_short_term_insight(content)
                insight_content.append(insight)

        # remove the transferred memories from short-term memories
        if transfer_flag:
            # re-construct the indexes of short-term memories after removing summarized memories
            new_memories = []
            new_embeddings = []
            new_importance = []
            new_enhance_memories = [[] for _ in range(self.capacity)]
            new_enhance_cnt = [0 for _ in range(self.capacity)]
            for idx, memory in enumerate(self.short_memories):
                if existing_memory[idx]:  # True
                    new_enhance_memories[len(new_memories)] = self.enhance_memories[idx]
                    new_enhance_cnt[len(new_memories)] = self.enhance_cnt[idx]
                    new_memories.append(memory)
                    new_embeddings.append(self.short_embeddings[idx])
                    new_importance.append(self.memory_importance[idx])
            self.short_memories = new_memories
            self.short_embeddings = new_embeddings
            self.memory_importance = new_importance
            self.enhance_memories = new_enhance_memories
            self.enhance_cnt = new_enhance_cnt

        if len(memory_content) > 0:
            # print("memory_content is not None")
            if len(memory_importance) == 0:
                memory_importance = [0.5]

        return memory_content, memory_importance, insight_content

    def discard_memories(self) -> str:
        """
        discard the least importance memory when short-term memory module exceeds its capacity
        :return: (str) The content of the discard memory
        """
        if len(self.short_memories) > self.capacity:
            memory_dict = dict()
            for idx in range(len(self.short_memories) - 1):
                memory_dict[self.short_memories[idx]] = {'enhance_count': self.enhance_cnt[idx],
                                                         'importance': self.memory_importance[idx]}

            sort_list = sorted(memory_dict.keys(),
                               key=lambda x: (memory_dict[x]['importance'], memory_dict[x]['enhance_count']))
            find_idx = self.short_memories.index(sort_list[0])
            self.enhance_cnt.pop(find_idx)
            self.enhance_cnt.append(0)
            self.enhance_memories.pop(find_idx)
            self.enhance_memories.append([])
            self.memory_importance.pop(find_idx)
            discard_memory = self.short_memories.pop(find_idx)
            self.short_embeddings.pop(find_idx)

            # remove the discard_memory from other short-term memory's enhanced list
            for idx in range(len(self.short_memories)):
                if self.enhance_memories[idx].count(sort_list[0]) != 0:
                    self.enhance_memories[idx].remove(sort_list[0])
                    self.enhance_cnt[idx] -= 1

            return discard_memory

    @staticmethod
    def cosine_similarity(embedding1: List[float], embedding2: List[float]):
        """
        Calculate the cosine similarity between two vectors.
        :param embedding1: the first embedding
        :param embedding2: the second embedding
        :return: (float) the cosine similarity
        """
        dot_product = np.dot(embedding1, embedding2)
        norm1 = np.linalg.norm(embedding1)
        norm2 = np.linalg.norm(embedding2)
        similarity = dot_product / (norm1 * norm2)
        return similarity


    def add_stm_memory(self, observation: str, importance: float, op: str):
        """
        Add a new observation into short-term memory, and return the enhanced short-term memory and with the insight.
        :param observation: the content of the sensory memory of retrieved memory
        :param imporatance: the importance score of observation
        :param op: specify the types of observation. "add" means that the observation is sensory memory,
                 "retrieval" means that the observation is the retrieved memory.
        """
        const = 0.1
        # compute the vector similarities between observation and the existing short-term memories
        embeddings_model = OpenAIEmbeddings()
        observation_embedding = embeddings_model.embed_query(observation)
        for idx, memory_embedding in enumerate(self.short_embeddings):
            similarity = self.cosine_similarity(observation_embedding, memory_embedding)
            # primacy effect
            # The following one line was corrected by Zeyu on 23.8.27-7pm. Ori: if idx + 1 == len(short_term_embeddings):
            if idx + 1 == len(self.short_embeddings):
                similarity += const
            # sample and select the enhanced short-term memory
            # Sigmoid function
            prob = 1 / (1 + np.exp(-similarity))
            if prob >= 0.7 and random.random() <= prob:
                self.enhance_cnt[idx] += 1
                self.enhance_memories[idx].append(observation)
        memory_content, memory_importance, insight_content = self.transfer_memories(observation)
        if op == "add":
            self.short_memories.append(observation)
            self.memory_importance.append(importance)
            self.short_embeddings.append(observation_embedding)
            self.discard_memories()
        return memory_content, memory_importance, insight_content

In [ ]:
import logging
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import BaseMemory
import random
import re
from typing import Any, Dict, List, Optional


import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from langchain.base_language import BaseLanguageModel
from langchain.prompts import PromptTemplate
from langchain.utils import mock_now

import os

from langchain.llms import OpenAI
from langchain.chains import LLMChain
logger = logging.getLogger(__name__)

# llm = OpenAI(model="gpt-3.5-turbo-instruct")

class Document:
    """Example Document class."""
    def __init__(self, page_content: str, metadata: dict = None):
        self.page_content = page_content
        if metadata is None:
            metadata = {}
        if "created_at" not in metadata:
            metadata["created_at"] = datetime.now()  # Set default created_at value
        if "last_accessed_at" not in metadata:
            metadata["last_accessed_at"] = datetime.now()  # Set default last_accessed_at value
        self.metadata = metadata

class TimeWeightedVectorStoreRetriever:
    """Example TimeWeightedVectorStoreRetriever class."""
    def __init__(self, now: datetime, memory_stream: List[Document], vectorstore: Any):
        self.now = now
        self.memory_stream = memory_stream
        self.vectorstore = vectorstore

class RecAgentRetriever(TimeWeightedVectorStoreRetriever):
    """
    RecAgentRetriever is to retrieve memories from long-term memory module based on memory salience, importance and recency.
    """
    def __init__(self, now: datetime, memory_stream: List[Document], k: int, vectorstore: Any):
        super().__init__(now, memory_stream, vectorstore)
        self.k = k
        self.default_salience = 0.5  # Add default_salience attribute

    def get_salient_docs(self, query: str) -> Dict[int, float]:
        """
        Return salient documents based on the query.

        This method calculates the salience score for each document in the memory stream
        based on the given query using TF-IDF cosine similarity.
        """
        salient_docs = {}

        # Filter out non-Document objects from memory_stream
        document_memory_stream = [doc for doc in self.memory_stream if isinstance(doc, Document)]

        # Extract text content from documents
        document_texts = [doc.page_content for doc in document_memory_stream]

        # Vectorize documents and query using TF-IDF
        vectorizer = TfidfVectorizer()
        document_vectors = vectorizer.fit_transform(document_texts)
        query_vector = vectorizer.transform([query])

        # Calculate cosine similarity between query and documents
        similarities = cosine_similarity(query_vector, document_vectors)[0]

        # Assign similarity scores to documents
        for i, doc in enumerate(document_memory_stream):
            # print("\nSimilarities check: ", doc.metadata, "similar:", similarities[i])
            # salient_docs[doc.metadata["buffer_idx"]] = similarities[i]
            salient_docs[doc.metadata["importance"]] = similarities[i]

        return salient_docs

    def add_documents(self, documents: List[Document], current_time: datetime):
        """
        Add new documents to the memory stream.
        """
        for document in documents:
            document.metadata["last_accessed_at"] = current_time
            self.memory_stream.append(document)
            # print("\nDocuments in add Document:", self.memory_stream, "Page content in add document:", document.page_content, "Metadata in add document:", document.metadata)

    def get_relevant_documents(self, query: str) -> List[Document]:
        """Return documents that are relevant to the query."""
        current_time = self.now
        relevant_docs = [doc for doc in self.memory_stream if isinstance(doc, Document)]  # Filter out non-Document objects
        docs_and_scores = {
            doc: self.default_salience
            # Calculate for all memories.
            for doc in relevant_docs
        }
        # If a doc is considered salient, update the salience score
        salient_docs = self.get_salient_docs(query)
        for doc in salient_docs:
            docs_and_scores[doc] = salient_docs[doc]
        rescored_docs = [
            (doc, relevance, 0.4)
            for doc, relevance in docs_and_scores.items() if isinstance(doc, Document)  # Filter out non-Document objects
        ]
        rescored_docs.sort(key=lambda x: x[2], reverse=True)
        result = []
        # Ensure frequently accessed memories aren't forgotten
        retrieved_num = 0
        for doc, _, _ in rescored_docs:
            if retrieved_num < self.k and doc.page_content.find('[FORGET]') == -1 \
                    and doc.page_content.find('[MERGE]') == -1:
                retrieved_num += 1
                # buffered_doc = self.memory_stream[doc.metadata['buffer_idx']]
                doc.metadata["last_accessed_at"] = current_time
                # buffered_doc.metadata["last_accessed_at"] = current_time
                # result.append(buffered_doc)
                result.append(doc)
                # print("Memory result:", result, "Metadata:", doc.page_content)

        return result


class LongTermMemory(BaseMemory):
    """
    Long-term memory is the memory base for the RecAgent.
    """
    llm: BaseLanguageModel
    now: datetime
    memory_retriever: RecAgentRetriever

    verbose: bool = False

    reflection_threshold: Optional[float] = None
    max_tokens_limit: int = 1000000000
    aggregate_importance: float = 0.0
    decay_rate: float = 0.01
    """The exponential decay factor used as (1.0-decay_rate)**(hrs_passed)."""

    # input keys
    queries_key: str = "queries"
    most_recent_memories_token_key: str = "recent_memories_token"
    add_memory_key: str = "add_memory"
    # output keys
    relevant_memories_key: str = "relevant_memories"
    relevant_memories_simple_key: str = "relevant_memories_simple"
    most_recent_memories_key: str = "most_recent_memories"
    now_key: str = "now"

    reflecting: bool = False
    forgetting: bool = False

    forget_num: int = 3

    importance_weight: float = 0.15
    """How much weight to assign the memory importance."""


    def chain(self, prompt: PromptTemplate) -> LLMChain:
        return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)

    @staticmethod
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r"\n", text.strip())
        return [re.sub(r"^\s*\d+\.\s*", "", line).strip() for line in lines]


    @staticmethod
    def _parse_insight_with_connections(text: str):
        """
    #     Parse the output of LLM to the insight and the corresponding connections.
    #     :param text: The output of LLM.
    #     :return: The insight, and the list of connections.
    #     """
        pattern = r'\[.*?\]'
        insight = re.sub(pattern, '', text)
        nums = re.findall(r'\d+', text)
        if len(nums) != 0:
            connection_list = list(map(int, nums))
        else:
            connection_list = [0]

        return insight, connection_list


    def _score_memory_importance(self, memory_content: str) -> float:
        """
        Obtain the importance score of this memory.
        :param memory_content: The text of the observation.
        :return: (float) The importance of this observation.
        """
        prompt = PromptTemplate.from_template(
            """
            Please give an importance score between 1 to 10 for the following observation. Higher score indicates the observation is more important. More rules that should be followed are
            \n(1) The observation that includes entering social media is not important. e.g., David Smith takes action by entering the world of social media.
            \n(2) The observation that describes chatting with someone but no specific book name is not important. e.g., David Smith observed that David Miller expressed interest in chatting about books.
            \n(3) The observation that includes 'chatting' is not important. e.g., David Smith observed that David Miller expressed interest in chatting about books, indicating a shared passion for books.
            \n(4) The observation that includes 'enter the recommender system' is not important. e.g. David Smith enters the Recommender System to explore book recommendations based on his interests and preferences.
            \n(5) The observation that recommends or mentions specific books is important.
            \n(6) More informative indicates more important, especially when two people are chatting.
            Please respond with a single integer.
            \nObservation:{memory_content}
            \nRating:
            """
        )
        score = LLMChain(llm=self.llm, prompt=prompt).run(memory_content=memory_content).strip()
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(match.group(1)) / 10) * self.importance_weight
        else:
            return 0.0

    def fetch_memories_with_list(self, observation, stm):
        """
        Transfer the retrieved memories and the enhanced short-term memory with the insight into List.
        :param observation: the observation to retrieve related memories
        :param stm: the short term memory instance
        :return
            (List[(float, str)]) res: the list tuples contains the memory content with corresponding importance score
            (Tuple(List[str], List[float], List[str])) memories_tuple: contains the short-term memories, importances and the insights.
        """
        res_list, memories_tuple = self.fetch_memories(observation, stm=stm)
        res = [(res.metadata['importance'], res.page_content) for res in res_list]
        return res, memories_tuple

    def fetch_memories(self, observation: str, stm=None, now: Optional[datetime] = None):
        """
        Fetch related memories.
        :param observation: the observation to retrieve related memories
        :param stm: the short term memory instance
        :param now: (optional) the current time.
        :return
                (List[Document]) the retrieved memory documents
                (Tuple(List[str], List[float], List[str])) memories_tuple: contains the short-term memories, importances and the insights.
        """
        with mock_now(now):
            # reflection do not enhance the short-term memories
            retrieved_list = self.memory_retriever.get_relevant_documents(observation)
            if stm is None:
                return retrieved_list
            # retrieval enhance the short-term memories
            else:
                ltm_memory_list, ltm_importance_scores = [], []
                insight_memory_list = []
                for document in retrieved_list:
                    memory_content, memory_importance, insight_content = \
                        stm.add_stm_memory(document.page_content, document.metadata['importance'], op='Retrieval')
                    ltm_memory_list.extend(memory_content)
                    ltm_importance_scores.extend(memory_importance)
                    insight_memory_list.extend(insight_content)

                for idx in range(len(stm.short_memories)):
                    short_term_document = Document(
                        page_content=stm.short_memories[idx],
                        metadata={"importance": stm.memory_importance[idx]}
                    )
                    retrieved_list.append(short_term_document)

                return retrieved_list, (ltm_memory_list, ltm_importance_scores, insight_memory_list)

    def format_memories_detail(self, relevant_memories: List[Document]) -> str:
        content_strs = set()
        content = []
        for mem in relevant_memories:
            if mem.page_content in content_strs:
                continue
            content_strs.add(mem.page_content)
            created_time = mem.metadata["created_at"].strftime("%B %d, %Y, %I:%M %p")
            content.append(f"- {created_time}: {mem.page_content.strip()}")
        return "\n".join([f"{mem}" for mem in content])


    def format_memories_simple(self, relevant_memories: List[Document]) -> str:
        return "; ".join([f"{mem.page_content}" for mem in relevant_memories])

    def _get_memories_until_limit(self, consumed_tokens: int) -> str:
        """Reduce the number of tokens in the documents."""
        result = []
        for doc in self.memory_retriever.memory_stream[::-1]:
            if consumed_tokens >= self.max_tokens_limit:
                break
            consumed_tokens += self.llm.get_num_tokens(doc.page_content)
            if consumed_tokens < self.max_tokens_limit:
                result.append(doc)
        result = self.format_memories_simple(result)
        return result

    @property
    def memory_variables(self) -> List[str]:
        return []

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        pass

    @staticmethod
    def cosine_similarity(embedding1: List[float], embedding2: List[float]):
        dot_product = np.dot(embedding1, embedding2)
        norm1 = np.linalg.norm(embedding1)
        norm2 = np.linalg.norm(embedding2)
        similarity = dot_product / (norm1 * norm2)
        return similarity

    def _get_topics_of_reflection(self, last_k: int = 50) -> List[str]:
        """Return the 1 most salient high-level questions about recent observations."""
        prompt = PromptTemplate.from_template(
            "{observations}\n\n"
            + "Given only the information above, what is the 1 most salient"
            + " high-level questions we can answer about the subjects in"
            + " the statements? Provide the question on a new line.\n\n"
        )
        observations = self.memory_retriever.memory_stream[-last_k:]
        observation_str = "\n".join([o.page_content for o in observations if hasattr(o, 'page_content')])
        result = self.chain(prompt).run(observations=observation_str)
        # print("\n_get_topics_of_reflection result:", result)
        return self._parse_list(result)

    def _get_insights_on_topic(
            self, topic: str, now: Optional[datetime] = None
    ):
        """Generate 'insights' on a topic of reflection, based on pertinent memories."""
        prompt_insight = PromptTemplate.from_template(
            "The topic or question is:\n"
            + "{topic}\n"
            + "Some statements are provided in the format [id] in the following:\n"
            + "{related_statements}\n"
            + "please identify one main insight to answer the above question, "
            + "and simultaneously specify which statements the insight is derived from:\n"
            + "Respond ONLY with the insight and the Ids of their related statements, adhering strictly to the following format:\n"
            + "Content of insight [Related statement IDs]\n"
            + "An insight can be derived from multiple statements."
            + "The insight needs to be significantly different from the statement(s) in sentence structure and content it is derived from."
        )

        related_memories = self.fetch_memories(topic, now=now)
        related_statements = "\n".join(
            [
                # f"{i + 1}. {memory.page_content}"
                f"{memory.page_content}"
                for i, memory in enumerate(related_memories)
            ]
        )

        result_insight = self.chain(prompt_insight).run(
            topic=topic, related_statements=related_statements
        )

        result_insight = self._parse_list(result_insight)
        result_insight = [self._parse_insight_with_connections(res) for res in result_insight]
        statements_id = result_insight[0][1]

        pattern = r"(?<=\[)\d+(?=\])"
        indexes = []
        embeddings_model = OpenAIEmbeddings()
        embedding_1 = embeddings_model.embed_query(result_insight[0][0])
        for memory_id in statements_id:
            if memory_id < 0 or memory_id >= len(self.memory_retriever.memory_stream):
                continue
            # print("\n_get_insights_on_topic: ", self.memory_retriever.memory_stream[memory_id])
            # print("\n_get_insights_on_topic page content: ", self.memory_retriever.memory_stream[memory_id].page_content)
            if hasattr(self.memory_retriever.memory_stream[memory_id], 'page_content'):
                memory = self.memory_retriever.memory_stream[memory_id].page_content
                if memory == '[MERGE]' or memory == '[FORGET]':
                    continue
                memory_embedding = embeddings_model.embed_query(memory)
                similarity = self.cosine_similarity(embedding_1, memory_embedding)
                # Sigmoid function
                value = 1 / (1 + np.exp(-similarity))
                if value >= 0.72:
                    match = re.search(pattern, memory)
                    idx = match.group()
                    indexes.append(int(idx))


        for idx in indexes:
            self.memory_retriever.memory_stream[idx].page_content = '[MERGE]'
            self.memory_retriever.memory_stream[idx].metadata['importance'] = 1.0
            self.memory_retriever.memory_stream[idx].metadata['last_accessed_at'] = self.now

        return result_insight

    def pause_to_reflect(self, now: Optional[datetime] = None):
        """
        Reflect on recent observations and generate 'insights'.
        :param now: (optional) The current time.
        :return: The list of new insights. [No use for this version.]
        """
        if self.verbose:
            logger.info("Character is reflecting")

        new_insights = []
        topics = self._get_topics_of_reflection()
        # print("\nTopics: ", topics)
        for topic in topics:
            insights = self._get_insights_on_topic(topic, now=now)
            for insight in insights:
                text, par_list = insight
                importance_cur, recency_cur = 0.0, 0.0
                valid = 0
                for par in par_list:
                    if par < len(self.memory_retriever.memory_stream):
                        if hasattr(self.memory_retriever.memory_stream[par], 'metadata'):
                            importance_cur += self.memory_retriever.memory_stream[par].metadata['importance']
                            valid += 1
                            buffer_idx = self.memory_retriever.memory_stream[par].metadata.get('buffer_idx', None)  # Get buffer_idx if available
                            if buffer_idx is None:
                                # Assign buffer_idx if it's not present
                                buffer_idx = len(self.memory_retriever.memory_stream) + 1
                                self.memory_retriever.memory_stream[par] = buffer_idx  # Update metadata
                if valid == 0:
                    importance_cur = 0.0
                else:
                    importance_cur /= valid
                ltm = importance_cur, now, text
                self.add_memory(ltm, now=now)
            new_insights.extend(insights)
        print("\nNew Insight Memory:", new_insights)

        return new_insights

    def obtain_forget_prob_list(self):
        """
        Obtain the forgetting probability of each memory.
        :return: (List) The distribution of forgetting probability.
        """

        def score_func(importance, last_accessed_time):
            """
            Given the importance score and last accessed time, calculate the score of this memory.
            :param importance: The importance score.
            :param last_accessed_time: The last accessed time.
            :return: Score of this memory.
            """
            hours_passed = (self.now - last_accessed_time).total_seconds() / 3600
            recency = (1.0 - self.decay_rate) ** hours_passed

            return max(recency ** 1.5, 0.01) * (importance + recency) / 2

        score_list = []
        for ind, mem in enumerate(self.memory_retriever.memory_stream):
            # print("........MEM>>>>>>>: ", mem)
            if hasattr(mem, 'metadata'):
                score = score_func(mem.metadata['importance'], mem.metadata['last_accessed_at'])
                score_list.append(score)
        score_list = 1.0 - np.array(score_list)
        return score_list / np.sum(score_list)

    def pause_to_forget(self):
        """
        Forget parts of long term memories.
        """
        if self.verbose:
            logger.info("Character is forgetting.")

        prob_list = self.obtain_forget_prob_list()
        i = 0
        if len(prob_list) != 0:
            for idx, memory in enumerate(self.memory_retriever.memory_stream):
                if hasattr(memory, 'metadata'):
                    if (self.now - self.memory_retriever.memory_stream[idx].metadata['last_accessed_at']).total_seconds() / 3600 <= 24:
                        continue
                    if random.random() < prob_list[i]:
                        self.memory_retriever.memory_stream[idx].page_content = '[FORGET]'
                        self.memory_retriever.memory_stream[idx].metadata['importance'] = 1.0
                        self.memory_retriever.memory_stream[idx].metadata['last_accessed_at'] = self.now
                    i=i+1

    def add_memory(self, ltm, now=None):
        """
        Store the long term memory.
        :param ltm: The long term memory that is ready to be stored.
        :param now: Current time.
        :return: List of IDs of the added texts. [No use in this version.]
        """
        importance, last_accessed_at, text = ltm
        if not self.reflecting:
            self.aggregate_importance += importance
        memory_idx = len(self.memory_retriever.memory_stream)
        document = Document(
            page_content='[{}] '.format(memory_idx) + str(text),
            metadata={"importance": importance, "last_accessed_at": last_accessed_at}
        )
        # print("\nDocuments: Page_content: ",document.page_content, ", Metadata: ", document.metadata)
        result = self.memory_retriever.add_documents([document], current_time=now)
        return result

    def save_context(self, inputs: Dict[str, Any], ltm_list: list) -> None:
        """
        Store the long term memories. Execute reflection and forgetting.
        :param inputs: [No use for this version.]
        :param ltm_list: The list of long term memory with tuple format (importance score[float], now[datetime], memory[string]).
        :return: None
        """

        now = self.now
        for ltm in ltm_list:
            self.add_memory(ltm, now)

        # When the aggregation of importance is above the threshold, execute the reflection function once.
        if (
                self.reflection_threshold is not None
                and self.aggregate_importance > self.reflection_threshold
                and not self.reflecting
        ):
            self.reflecting = True
            self.pause_to_reflect(now=now)
            # Hack to clear the importance from reflection
            self.aggregate_importance = 0.0
            self.reflecting = False

        # Execute the forget function once.
        if True:
            self.forgetting = True
            self.pause_to_forget()
            self.forgetting = False

    def print_memory(self):
        """
        [Tool for Debug] Print the long term memories.
        """
        for ind, mem in enumerate(self.memory_retriever.memory_stream):
            if hasattr(mem, 'metadata'):
                hours_passed = (self.now - mem.metadata['last_accessed_at']).total_seconds() / 3600
                recency = (1.0 - self.decay_rate) ** hours_passed
                # uncomment this line when you want to check the memory contents
                # print('[%d] (importance: %f, recency: %f) %s' % (
                #     ind, mem.metadata['importance'], recency, mem.page_content))

    def update_now(self, now: datetime):
        """
        Update the current time.
        :param now: Current time.
        """
        self.now = now
        self.memory_retriever.now = now
        meminer = []
        for memo in self.memory_retriever.memory_stream:
            if hasattr(memo, 'metadata'):
                meminer.append(memo.page_content)
        print("\n\nLong Term Memory Content :", meminer)

    def clear(self) -> None:
        """
        Clear all the memories in long term memory.
        """
        self.memory_retriever.memory_stream = []

In [ ]:
import re
from copy import deepcopy
from datetime import datetime
from typing import Any, Dict, List, Optional

from langchain import LLMChain
from langchain.base_language import BaseLanguageModel
from langchain.prompts import PromptTemplate
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import BaseMemory


class RecAgentMemory(BaseMemory):
    """
    RecAgentMemory is the proposed memory module for RecAgent. We replace `GenerativeAgentMemory` with this class.
    Similarly, it has three necessary methods to implement:
    - load_memory_variables: given inputs, return the corresponding information in the memory.
    - save_context: accept observations and store them as memory.
    - clear: clear the memory content.

    We have three key components, which is consistent with human's brain.
    - SensoryMemory: Receive observations, abstract significant information, and pass to short-term memory.
    - ShortTermMemory: Receive sensory memories, enhance them with new observations or retrieved memories,
                       and then transfer the enhanced short-term memories with an insight to long-term memory,
                       or discard the less important memory in cases of capacity overload.
    - LongTermMemory: Receive short-term memories, store and forget memories, and retrival memories to short-term memory.

    """
    llm: BaseLanguageModel = None
    verbose: bool = False
    now: datetime = None

    # Comment those lines when you don't want to use the memory module
    sensoryMemory: SensoryMemory = None
    shortTermMemory: ShortTermMemory = None
    longTermMemory: LongTermMemory = None

    importance_weight: float = 0.9
    """How much weight to assign the memory importance."""

    # input keys
    queries_key: str = "queries"
    most_recent_memories_token_key: str = "recent_memories_token"
    add_memory_key: str = "add_memory"
    # output keys
    relevant_memories_key: str = "relevant_memories"
    relevant_memories_simple_key: str = "relevant_memories_simple"
    most_recent_memories_key: str = "most_recent_memories"
    now_key: str = "now"

    def __init__(self, llm, memory_retriever, now, verbose=False, reflection_threshold=None):
        super(RecAgentMemory, self).__init__()

        self.llm = llm
        self.now = now
        # Comment those lines when you don't want to use the memory module
        self.sensoryMemory = SensoryMemory(llm)
        self.shortTermMemory = ShortTermMemory(llm)
        self.longTermMemory = LongTermMemory(llm=llm, memory_retriever=memory_retriever, now=self.now, verbose=verbose,
                                             reflection_threshold=reflection_threshold)

    def chain(self, prompt: PromptTemplate) -> LLMChain:
        return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)

    @property
    def memory_variables(self) -> List[str]:
        return []

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """
        Return 'most_recent_memories' with fetched memories (not recent memories).
        :param inputs: The dict that contains the key 'observation'.
        :return: The fetched memories.
        """

        ltm_memory_list, memories_tuple = self.longTermMemory.fetch_memories_with_list(inputs['observation'],
                                                                                       self.shortTermMemory)
        self.save_context_after_retrieval(memories_tuple)
        if len(ltm_memory_list) == 0:
            memory_tmp = ''
        else:
            memory_tmp = [memory[1] for memory in ltm_memory_list]
        memory_tmp = ''.join(memory_tmp)
        # memory_tmp = '' #unused line

        output = {'most_recent_memories': memory_tmp}
        return output

    def _score_memory_importance(self, memory_content: str) -> float:
        """Score the absolute importance of the given memory."""
        prompt = PromptTemplate.from_template(
            """
            Please give an importance score between 1 to 10 for the following observation. Higher score indicates the observation is more important. More rules that should be followed are
            \n(1) The observation that includes entering social media is not important. e.g., David Smith takes action by entering the world of social media.
            \n(2) The observation that describes chatting with someone but no specific book title is not important. e.g., David Smith observed that David Miller expressed interest in chatting about books.
            \n(3) The observation that includes 'chatting' is not important. e.g., David Smith observed that David Miller expressed interest in chatting about books, indicating a shared passion for books.
            \n(4) The observation that includes 'enter the recommender system' is not important. e.g. David Smith enters the Recommender System to explore book recommendations based on his interests and preferences.
            \n(5) The observation that recommends or mentions specific books is important.
            \n(6) More informative indicates more important, especially when two people are chatting.
            Please respond with a single integer.
            \nObservation:{memory_content}
            \nRating:
            """
        )
        score = self.chain(prompt).run(memory_content=memory_content).strip()
        if self.verbose:
            logger.info(f"Importance score: {score}")
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(match.group(1)) / 10) * self.importance_weight
        else:
            return 0.0

    def add_memory(self, memory_content: str, now: Optional[datetime] = None):
        """
        The Simulator can add memory by using this function.
        :param memory_content: The content of memory.
        :param now: Current time.
        """
        self.save_context(
            {},
            {
                self.add_memory_key: memory_content,
                self.now_key: now,
            },
        )

    def save_context_after_retrieval(self, memories_tuple):
        """
        The RecAgent can transfer short-term memory to long-term memory.
        :param memories_tuple:  (Tuple(List[str], List[float], List[str])) memories_tuple: contains the short-term memories, importances and the insights.
        :return None
        """
        ltm_memory_list, ltm_importance_scores, insight_memory_list = memories_tuple
        insight_memory_list = [memory[0] for memory in insight_memory_list]
        insight_scores_list = [self._score_memory_importance(memory) for memory in insight_memory_list]

        all_memories = ltm_memory_list + insight_memory_list
        all_memory_scores = ltm_importance_scores + insight_scores_list
        save_ltm_memory = [(all_memory_scores[i], self.now, all_memories[i])
                           for i in range(len(all_memories))]
        self.longTermMemory.save_context({}, save_ltm_memory)

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """
        The RecAgent can add memory by using this function.
        :param inputs: Will be directly pass to LongTermMemory. [No use for this version.]
        :param outputs: The core memory dict that is passed from RecAgent. It has to obtain the key 'add_memory' to save the memory content.
        :return: None
        """
        # If the 'outputs' does not contain the memory, then exit the function.
        # print("Entering Memory.............\n\n")
        if 'add_memory' not in outputs:
            return
        # Comments all those lines when you don't want to use memory module
        # Add the observation into the buffer of sensory memory, and obtain a list of short term memory if the buffer is full.
        obs = outputs['add_memory']
        stm_memory_list = self.sensoryMemory.add_ssm(obs)
        # print("stm_memory_list:", stm_memory_list)
        if stm_memory_list is None:
            return
        else:
            ltm_memory_list, ltm_importance_scores = [], []
            insight_memory_list = []
            for stm_memory in stm_memory_list:
                memory_content, memory_importance, insight_content \
                    = self.shortTermMemory.add_stm_memory(stm_memory[1], stm_memory[0], op='add')
                # print("\nmemory_content:", memory_content)
                # print("\nmemory_importance:", memory_importance)
                # print("\ninsight_content:", insight_content)
                ltm_memory_list.extend(memory_content)
                ltm_importance_scores.extend(memory_importance)
                insight_memory_list.extend(insight_content)

            insight_memory_list = [memory[0] for memory in insight_memory_list]
            insight_scores_list = [self._score_memory_importance(memory) for memory in insight_memory_list]

            all_memories = ltm_memory_list + insight_memory_list
            all_memory_scores = ltm_importance_scores + insight_scores_list
            # print("\nall_memories:", all_memories)
            # print("\nall_memory_scores:", all_memory_scores)
            # Ensure that all lists have the same length before creating save_ltm_memory
            if len(all_memories) == len(all_memory_scores):
                save_ltm_memory = [(all_memory_scores[i], self.now, all_memories[i])
                       for i in range(len(all_memories))]
                # Store the long term memories.
                self.longTermMemory.save_context(inputs, save_ltm_memory)
            else:
                # Handle the case where the lists have different lengths
                print("Error: Lists have different lengths.")


    def update_now(self, now: datetime):
        """
        Update the current time.
        :param now: Current time.
        """
        self.now = now
        # Make comments when you don't want to use the memory module
        self.longTermMemory.update_now(self.now)


    def clear(self) -> None:
        """
        Clear all the (long term) memory in RecAgentMemory.
        """
        # Make comments when you don't want to use the memory module
        self.longTermMemory.clear()
        # self.clear() # unused line


In [ ]:
import asyncio
from datetime import datetime

class RoleAgent:
    run_location: str = "web"

    def __init__(self, id, name, age, gender, traits, status, interest, relationships, feature, memory_retriever, llm, memory, event, avatar_url, idle_url, watching_url, chatting_url, posting_url):
        self.id = id
        self.name = name
        self.age = age
        self.gender = gender
        self.traits = traits
        self.status = status
        self.interest = interest
        self.relationships = relationships
        self.feature = feature
        self.memory_retriever = memory_retriever
        self.llm = llm
        self.memory = memory
        self.event = event
        self.avatar_url = avatar_url
        self.idle_url = idle_url
        self.watching_url = watching_url
        self.chatting_url = chatting_url
        self.posting_url = posting_url
        self.role = "user"

    @classmethod
    def from_recagent(cls, recagent_instance):
        return cls(**recagent_instance.__dict__)

    async def get_response(self, message):
        print("Received message:", message)
        response = input("Enter your response: ")
        return response

    async def take_action(self, now):
        order = await self.get_response(
            f"It's {now}.\n"
            "Please choose one action below: \n"
            "(1) Enter the Recommender, please input 1. \n"
            "(2) Enter the Social Media, please input 2. \n"
            "(3) Do Nothing, please input 3. \n")
        while order not in ["1", "2", "3"]:
            order = await self.get_response(
                "Your input is wrong, please choose one action below: \n"
                "(1) Enter the Recommender, please input 1. \n"
                "(2) Enter the Social Media, please input 2. \n"
                "(3) Do Nothing, please input 3. \n")

        action = await self.get_response("You can input some text to explain your choice. \n")
        choice = {"1": "[RECOMMENDER]", "2": "[SOCIAL]", "3": "[NOTHING]"}[order]
        phase = {
            "1": "enter the Recommender System",
            "2": "enter the Social Media",
            "3": "do nothing",
        }[order]
        result = choice + ":: %s wants to %s because %s" % (self.name, phase, action)

        return choice, result

    async def take_recommender_action(self, observation, now):
        """
        Require the user choose one action below by inputting:
        (1) Buy movies among the recommended items.
        (2) Next page.
        (3) Search items.
        (4) Leave the recommender system.
        :return
        choice(str): the token that represents the choice made by the user, one of in '[BUY]', '[NEXT]',
                     '[SEARCH]', and '[NOTHING]'.
        action(str): integrate the choice and the reason into one sentence.
        """
        order = await self.get_response(
            observation+
            "\nPlease choose one action below: \n"
            "(1) Buy movies among the recommended items, please input 1. \n"
            "(2) Next page, please input 2. \n"
            "(3) Search items, please input 3. \n"
            "(4) Leave the recommender system, input 4. \n"
        )

        # If the input is not conforming to the prescribed form, we let the user input again.
        while order not in ["1", "2", "3", "4"]:
            order = await self.get_response(
                "Your input is wrong, please choose one action below: \n"
                "(1) Buy movies among the recommended items, please input 1. \n"
                "(2) Next page, please input 2. \n"
                "(3) Search items, please input 3. \n"
                "(4) Leave the recommender system, input 4. \n"
            )

        # Change the input number to the choice token.
        choice = {"1": "[BUY]", "2": "[NEXT]", "3": "[SEARCH]", "4": "[LEAVE]"}[order]

        if order == "1":
            films = await self.get_response(
                "Please input movie names in the list returned by the recommender system, only movie names, separated by semicolons. \n"
            )
            # Construct the list of films with '<*>' format.
            film_list = ["<%s>" % film for film in films.split(",")]
            action = str(film_list)
        elif order == "2":
            action = f"{self.name} looks at the next page"
        elif order == "3":
            items = await self.get_response("Please input the single, specific item name you want to search. \n")
            # Construct the list of items with '<*>' format.
            item_list = ["<%s>" % item for item in items.split(",")]
            action = str(item_list)
        elif order == "4":
            action = f"{self.name} leaves the recommender system"
        else:
            raise ValueError("Unexpected order value.")

        result = f"{choice}:: {action}."

        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} took action: {result}",
            },
        )
        return choice, action


    async def generate_feeling(self, observation: str, now) -> str:
        feeling = await self.get_response("Please input your feelings, which should be split by semicolon: \n")

        results = feeling.split(",")
        feelings = "".join(results)
        self.memory.save_context({}, {self.memory.add_memory_key: f"{self.name} felt: {feelings}"})
        return feelings


    async def search_item(self, observation: str, now) -> str:
        search = await self.get_response("Please input your search: \n")
        self.memory.save_context({}, {self.memory.add_memory_key: f"{self.name} wants to search and watch {search} in recommender system."})
        return search


    async def take_social_action(self, observation: str, now) -> Tuple[str, str]:
        order = await self.get_response(
            observation +
            "\nPlease choose one action below: \n"
            "(1) Chat with one acquaintance, input 1. \n"
            "(2) Publish posting to all acquaintances, input 2. \n"
        )

        while order not in ["1", "2"]:
            order = await self.get_response(
                "Please choose one action below: \n"
                "(1) Chat with one acquaintance, input 1. \n"
                "(2) Publish posting to all acquaintances, input 2. \n"
            )

        choice = {"1": "[CHAT]", "2": "[POST]"}[order]

        if order == "1":
            action = await self.get_response("Please input one acquaintance to chat: \n")
        elif order == "2":
            action = " "
        else:
            raise ValueError("Unexpected order value.")

        result = f"{choice}:: {action}"
        self.memory.save_context({}, {self.memory.add_memory_key: f"{self.name} took action: {result}"})
        return choice, action

    async def generate_role_dialogue(self, agent2, observation, now=None):
        role_text = await self.get_response(
            'Please input your chatting text (Input "goodbye" if you want to quit): \n'
        )
        role_dia = "%s said %s" % (self.name, role_text)
        contin = True

        if role_text == "goodbye":
            contin = False

        return contin, role_dia

In [ ]:
from datetime import datetime

class SocialNetwork:
    def __init__(self):
        self.agents = {}  # agent_id -> agent info (dict)
        self.posts = []   # list of all posts

    def add_agent(self, agent_id, agent_name, interests=None):
        """Add an agent to the social network."""
        if agent_id in self.agents:
            print(f"Agent ID {agent_id} already exists.")
            return
        self.agents[agent_id] = {
            'agent_name': agent_name,
            'interests': set(interests) if interests else set()
        }


    def create_post(self, agent_id, round_number, public_posting, summary, book_genre):
        """Agent creates a post in the social network."""
        if agent_id not in self.agents:
            print(f"Agent ID {agent_id} does not exist.")
            return
        post = {
            'agent_id': agent_id,
            'agent_name': self.agents[agent_id]['agent_name'],
            'round_number': round_number,
            'public_posting': public_posting,
            'summary': summary,
            'book_genre': book_genre,
            'timestamp': datetime.now()
        }
        self.posts.append(post)
        print(self.format_post(post))

    def get_all_posts_except(self, agent_id):
        """Return all posts except those from the given agent."""
        return [post for post in self.posts if post['agent_id'] != agent_id]

    def get_ranked_posts_for_agent(self, agent_id):
        """Return posts ranked for a given agent based on overlapping genres and recency."""
        if agent_id not in self.agents:
           print(f"Agent ID {agent_id} does not exist.")
           return []

        # Normalize agent interests to lowercase
        interests = set(i.strip().lower() for i in self.agents[agent_id]['interests'])

        def post_score(post):
            score = 0
            # Ensure book_genre is a list
            book_genres = post['book_genre']
            if isinstance(book_genres, str):
               # if stored as comma-separated string, split:
               book_genres = [g.strip() for g in book_genres.split(',')]
            book_genres = set(g.strip().lower() for g in book_genres)

            # Count genre overlap
            genre_overlap = interests.intersection(book_genres)

            if genre_overlap:
               score += 2  # or use score += len(genre_overlap) for proportional bonus

            # Recency bonus
            days_since = (datetime.now() - post['timestamp']).days

            if days_since <= 7:
               score += 1

            return score, post['timestamp']

        posts = self.get_all_posts_except(agent_id)

        ranked_posts = sorted(
            posts,
            key=lambda post: (post_score(post)[0], post_score(post)[1]),
            reverse=True
            )
        return ranked_posts

    @staticmethod
    def format_post(post):
        return (
            f"Agent ID: {post['agent_id']}    Agent Name: {post['agent_name']}\n"
            f"Round: {post['round_number']}    Time: {post['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
            f"Public Posting: {post['public_posting']}    Genre: {post['book_genre']}\n"
            f"Summary: {post['summary']}\n"
        )

In [ ]:
import re
from datetime import datetime
from typing import Any, Dict, List, Optional, Tuple, ClassVar

from langchain_experimental.generative_agents.memory import (
    GenerativeAgentMemory,
    BaseMemory,
)
from langchain.prompts import PromptTemplate
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

from datetime import datetime
import os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI


class RecAgent(GenerativeAgent):
    id: int
    """The agent's unique identifier"""

    gender: str
    """The agent's gender"""

    traits: str
    """The agent's traits"""

    interest: str
    """The agent's movie interest"""

    feature: str
    """The agent's action feature"""


    relationships: Dict[str, str] = {}
    """The agent's relationship with other agents"""

    watched_history: List[str] = []
    """The agent's history of watched movies"""

    heared_history: List[str] = []
    """The agent's history of heared movies"""

    BUFFERSIZE: ClassVar[int] = 10
    """The size of the agent's history buffer"""

    max_dialogue_token_limit: int = 600
    """The maximum number of tokens to use in a dialogue"""

    event: Event
    """The agent action"""

    active_prob: float = 0.5
    """The probability of the agent being active"""

    no_action_round: int = 0
    """The number of rounds that the agent has not taken action"""

    memory: BaseMemory
    """The memory module in RecAgent."""

    role: str = "agent"

    avatar_url: str

    idle_url: str

    watching_url: str

    chatting_url: str

    posting_url: str

    @classmethod
    def from_roleagent(cls, roleagent_instance: "RecAgent"):
        # 使用RoleRecAgent实例的属性来创建一个RecAgent实例
        new_instance = cls(
            id=roleagent_instance.id,
            name=roleagent_instance.name,
            age=roleagent_instance.age,
            gender=roleagent_instance.gender,
            traits=roleagent_instance.traits,
            status=roleagent_instance.status,
            interest=roleagent_instance.interest,
            relationships=roleagent_instance.relationships,
            feature=roleagent_instance.feature,
            memory_retriever=roleagent_instance.memory_retriever,
            llm=roleagent_instance.llm,
            memory=roleagent_instance.memory,
            event=roleagent_instance.event,
            avatar_url=roleagent_instance.avatar_url,
            idle_url=roleagent_instance.idle_url,
            watching_url=roleagent_instance.watching_url,
            chatting_ulr=roleagent_instance.chatting_url,
            posting_url=roleagent_instance.posting_url,
        )
        return new_instance

    def __lt__(self, other: "RecAgent"):
        return self.event.end_time < other.event.end_time

    def chain(self, prompt: PromptTemplate) -> LLMChain:
        return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)

    def get_active_prob(self, method) -> float:
        if method == "marginal":
            return self.active_prob * (self.no_action_round + 1)
        else:
            return self.active_prob

    def update_from_dict(self, data_dict: dict):
        for key, value in data_dict.items():
            setattr(self, key, value)

    def get_summary(
        self,
        now: Optional[datetime] = None,
        observation: str = None,
    ) -> str:
        """Return a descriptive summary of the agent."""
        prompt = PromptTemplate.from_template(
            "Given the following observation about {agent_name}: '{observation}', please summarize the relevant details from his profile. His profile information is as follows:\n"
            + "Name: {agent_name}\n"
            + "Age: {agent_age}\n"
            + "Gender:{agent_gender}\n"
            + "Traits: {agent_traits}\n"
            + "Status: {agent_status}\n"
            + "Book Interest: {agent_interest}\n"
            + "Feature: {agent_feature}\n"
            + "Interpersonal Relationships: {agent_relationships}\n"
            + "Please avoid repeating the observation in the summary.\nSummary:"
        )
        kwargs: Dict[str, Any] = dict(
            observation=observation,
            agent_name=self.name,
            agent_age=self.age,
            agent_gender=self.gender,
            agent_traits=self.traits,
            agent_status=self.status,
            agent_interest=self.interest,
            agent_feature=self.feature,
            agent_relationships=self.relationships,
        )

        result = self.chain(prompt=prompt).run(**kwargs).strip()

        age = self.age if self.age is not None else "N/A"
        return (
            f"Name: {self.name} (age: {age}) Interest: {self.interest}" + f"\n{result}"
        )

    def _generate_reaction(
        self, observation: str, suffix: str, now: Optional[datetime] = None
    ) -> str:
        """React to a given observation."""
        prompt = PromptTemplate.from_template(
            "{agent_summary_description}"
            + "\nIt is {current_time}."
            + "\n{agent_name} recently heared {heared_history} on social media."
            + "\n{agent_name} recently read {watched_history} on recommender system."
            + "\nOther than that {agent_name} doesn't know any books."
            + "\nMost recent observations: {most_recent_memories}"
            + "\nObservation: {observation}"
            + "\nAll occurrences of book names should be enclosed with <>"
            + "\n\n"
            + suffix
        )
        now = datetime.now() if now is None else now
        agent_summary_description = self.get_summary(now=now, observation=observation)

        current_time_str = (
            datetime.now().strftime("%B %d, %Y, %I:%M %p")
            if now is None
            else now.strftime("%B %d, %Y, %I:%M %p")
        )
        kwargs: Dict[str, Any] = dict(
            agent_summary_description=agent_summary_description,
            current_time=current_time_str,
            agent_name=self.name,
            observation=observation,
            watched_history=(
                self.watched_history if len(self.watched_history) > 0 else "nothing"
            ),
            heared_history=(
                self.heared_history if len(self.heared_history) > 0 else "nothing"
            ),
        )
        consumed_tokens = self.llm.get_num_tokens(
            prompt.format(most_recent_memories="", **kwargs)
        )
        kwargs[self.memory.most_recent_memories_key] = consumed_tokens
        result = self.chain(prompt=prompt).run(**kwargs).strip()
        # print("\n\n Agent Result Description: ", result)
        return result

    def _generate_reaction_bewteen_two(
        self,
        agent2: "RecAgent",
        observation: str,
        suffix: str,
        now: Optional[datetime] = None,
    ) -> str:
        """React to a given observation or dialogue act."""
        prompt = PromptTemplate.from_template(
            "{agent_summary_description}"
            + "\n {agent_summary_description2}"
            + "\nIt is {current_time}."
            + "\n{agent_name} recently heared {heared_history} on social media."
            + "\n{agent_name} recently read {watched_history} on recommender system."
            + "\nOther than that {agent_name} doesn't know any books."
            + "\n{agent_name} would only mention the books had recently read, but not any other books."
            + "\n{agent_name2} recently heared {heared_history2} on social media."
            + "\n{agent_name2} recently read {watched_history2} on recommender system."
            + "\nOther than that {agent_name2} doesn't know any books."
            + "\n{agent_name} would only mention the books had recently read, but not any other books."
            + "\nMost recent observations of {agent_name}: {most_recent_memories}"
            + "\nMost recent observations of {agent_name2}: {most_recent_memories2}"
            + "\nObservation: {observation}"
            + "\nAll occurrences of book names should be enclosed with <>"
            + "\n\n"
            + suffix
        )
        now = datetime.now() if now is None else now
        agent_summary_description = self.get_summary(now=now, observation=observation)
        # print("\n_generate_reaction_bewteen_two: agent_summary_description: ", agent_summary_description)
        agent_summary_description2 = agent2.get_summary(
            now=now, observation=observation
        )
        # print("\n_generate_reaction_bewteen_two: agent2_summary_description: ", agent_summary_description2)
        current_time_str = (
            datetime.now().strftime("%B %d, %Y, %I:%M %p")
            if now is None
            else now.strftime("%B %d, %Y, %I:%M %p")
        )
        kwargs: Dict[str, Any] = dict(
            agent_summary_description=agent_summary_description,
            current_time=current_time_str,
            agent_name=self.name,
            observation=observation,
            watched_history=(
                self.watched_history if len(self.watched_history) > 0 else "nothing"
            ),
            heared_history=(
                self.heared_history if len(self.heared_history) > 0 else "nothing"
            ),
            agent_summary_description2=agent_summary_description2,
            agent_name2=agent2.name,
            watched_history2=(
                agent2.watched_history if len(agent2.watched_history) > 0 else "nothing"
            ),
            heared_history2=(
                agent2.heared_history if len(agent2.heared_history) > 0 else "nothing"
            ),
        )

        try:
            (
                result_memories2,
                memories_tuple,
            ) = agent2.memory.longTermMemory.fetch_memories_with_list(
                observation, agent2.memory.shortTermMemory
            )
            result_memories2 = [memory[1] for memory in result_memories2]
            most_recent_memories2 = '; '.join(result_memories2)
            agent2.memory.save_context_after_retrieval(memories_tuple)
        except:
            most_recent_memories2 = ""

        kwargs["most_recent_memories2"] = most_recent_memories2
        consumed_tokens = self.llm.get_num_tokens(
            prompt.format(most_recent_memories="", **kwargs)
        )
        kwargs[self.memory.most_recent_memories_key] = consumed_tokens
        result = self.chain(prompt=prompt).run(**kwargs).strip()
        return result

    def get_memories_until_limit(self, consumed_tokens: int) -> str:
        """Reduce the number of tokens in the documents."""
        retriever = (
            self.memory.longTermMemory.memory_retriever
            if type(self.memory) == RecAgentMemory
            else self.memory.memory_retriever
        )
        result = []
        for doc in retriever.memory_stream[::-1]:
            if consumed_tokens >= self.max_dialogue_token_limit:
                break
            consumed_tokens += self.llm.get_num_tokens(doc.page_content)
            if consumed_tokens < self.max_dialogue_token_limit:
                result.append(doc)
        if type(self.memory) == RecAgentMemory:
            result = self.memory.longTermMemory.format_memories_simple(result)
        else:
            result = self.memory.format_memories_simple(result)
        return result

    def take_action(self, now) -> Tuple[str, str]:
        """Take one of the actions below.
        (1) Enter the Recommender.
        (2) Enter the Social Media.
        (3) Do Nothing.
        """
        call_to_action_template = (
            "What action would {agent_name} like to take? Respond in one line."
            + "\nIf {agent_name} wants to enter the Recommender System, write:\n [RECOMMENDER]:: {agent_name} enters the Recommender System"
            + "\nIf {agent_name} wants to enter the Social Media, write:\n [SOCIAL]:: {agent_name} enters the Social Media"
            + "\nIf {agent_name} wants to do nothing, write:\n [NOTHING]:: {agent_name} does nothing"
        )
        observation = f"{self.name} must take only ONE of the actions below:(1) Enter the Recommender System. If so, {self.name} will be recommended some books, from which {self.name} can read some books, or search for books by himself.\n(2) Enter the Social Media. {self.name} can chat with friends or publish a post to all friends of {self.name}. If {self.name} recently read some books they might want to share with others.\n(3) Do Nothing."

        full_result = self._generate_reaction(observation, call_to_action_template, now)
        result = full_result.strip().split("\n")[0]

        choice = result.split("::")[0]
        # action = result.split("::")[1]

        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} take action: " f"{result}",
                self.memory.now_key: now,
            },
        )

        return choice, result

    def positive_behavior_update(self, observation: str, now) -> str:
        """React to a given observation."""
        prompt = PromptTemplate.from_template(
            "Agent Name: {agent_name}\n"
            + "Observation: {observation}\n"
            + "Recently Read Book Titles: {read_history}\n"
            + "Recently Heard Book Titles from Social Media: {heard_history}\n"
            + "Current Positive Behavioral Preferences: {agent_pos}\n"
            + "Analyze the agent's recently read and heard book titles. Identify any common authors, content themes (e.g., romance, adventure, political), popularity status, or average historical ratings of these two lists.\n"
            + "Provide ONLY a concise summary of the updated positive behavioral preference profile based on your analysis."
            )

        kwargs: Dict[str, Any] = dict(
            agent_name=self.name,
            observation=observation,
            agent_pos=self.positive_behavior,
            read_history=(
                self.watched_history if len(self.watched_history) > 0 else "nothing"
            ),
            heard_history=(
                self.heared_history if len(self.heared_history) > 0 else "nothing"
            ),
        )

        positive_behavior = self.chain(prompt=prompt).run(**kwargs).strip()
        print("positive behavior: ", positive_behavior)

        return positive_behavior


    def take_recommender_action(self, observation, now) -> Tuple[str, str]:
        """Take one of the four actions below.
        (1) Read books among the recommended items.
        (2) Next page.
        (3) Search items.
        (4) Leave the recommender system.
        """
        call_to_action_template = (
            "{agent_name} must choose one of the four actions below:\n"
            "(1) Read ONLY ONE book from the list returned by the recommender system.\n"
            "(2) See the next page.\n"
            "(3) Search for a specific item.\n"
            "(4) Leave the recommender system."
            + "\nIf {agent_name} has recently heard about a particular book on social media, {agent_name} usually want to search for that book on the recommender system."
            + "\nTo read a book from the recommended list that match {agent_name}'s interests, write:\n[BUY]:: Index of the book starting from 1 (e.g., [BUY]:: 3)"
            + "\nTo see the next page, write:\n[NEXT]:: {agent_name} views the next page."
            + "\nTo search for a specific item, write:\n[SEARCH]:: single, specific book name to search for."
            + "\nTo leave the recommender system, write:\n[LEAVE]:: {agent_name} leaves the recommender system."
        )
        full_result = self._generate_reaction(observation, call_to_action_template, now)

        result = full_result.strip()
        if result.find("::") != -1:
            choice, action = result.split("::")
            choice = choice.strip()
            match = re.search(r'(\d+)', action)

            if match:
                num = int(match.group(1))
                if 1 <= num <= 5:
                    action = num
                else:
                    action = 1

        else:
            choice = "[LEAVE]"
            action = f"{self.name} leaves the recommender system."
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} took action: {result}",
                self.memory.now_key: now,
            },
        )

        return choice, action

    def generate_feeling(self, observation: str, now) -> str:
        """Feel about each item bought."""
        call_to_action_template = (
            "{agent_name}, how did you feel about the book you just read? Describe your feelings in one line."
            + "NOTE: Please answer in the first-person perspective."
            + "\n\n"
        )

        full_result = self._generate_reaction(observation, call_to_action_template, now)
        results = [result.strip() for result in re.split(r'[.,]', full_result)]
        feelings = ""
        for result in results:
            if "language model" in result:
                break
            feelings += result + ". "

        # Remove trailing space and period
        feelings = feelings.strip(". ")

        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} felt: " f"{feelings}",
                self.memory.now_key: now,
            },
        )

        return feelings

    def generate_rec_rating(self, observation: str, now) -> str:

        call_to_action_template = (
        "Assuming you are {agent_name}. After interacting with the book the feeling are generated. "
        "Based on user profile and generated feelings, please very critically rate this book on a scale from 1 to 5, where 1 means you really dislike it and 5 means you really like it. "
        "Consider that your rating should critically reflect both your profile and immediate feelings generated for this book. If your generated feeling is less positive then the rating will be generally low, your score might also be lower. "
        "Please provide your rating in the following format: 'Rating: [Your Score]'.\n\n")
        full_result = self._generate_reaction(observation, call_to_action_template, now)
        rating_regex = r"Rating: (\d)"
        extracted_rating = re.search(rating_regex, full_result)
        rating = 0
        if extracted_rating:
            rating = extracted_rating.group(1)
            # print(f"Extracted Rating: {rating}")
        return rating

    def search_item(self, observation, now) -> str:
        """Search item by the item name."""

        call_to_action_template = (
            "If you were {agent_name}, what books would you be interested in and search for in the system? Respond only a single name you want to search and read in {heared_history}."
            + "\n\n"
        )

        full_result = self._generate_reaction(observation, call_to_action_template, now)
        result = full_result.strip().split("\n")[0]
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} wants to search and read {result} in recommender system.",
                self.memory.now_key: now,
            },
        )
        return result

    def take_social_action(self, observation, now) -> Tuple[str, str]:
        """Take one of the four actions below.
        (1) Chat with one acquaintance. [CHAT]:: TO [acquaintance]: what to say.
        (2) Publish posting to all acquaintances. [POST]:: what to say.
        """
        call_to_action_template = (
            "{agent_name} must take one of the two actions below:\n(1)Chat with one acquaintance about movies recently watched on recommender system: {watched_history}, or movies heared about on social media: {heared_history}.\n(2) Publish posting to all acquaintances about movies recently watched on recommender system: {watched_history}, or heared about on social media: {heared_history}."
            + "\nWhat action would {agent_name} like to take and how much time does the action cost?"
            + "\n{agent_name} should chat with more different people. If {agent_name} want to chat with one acquaintance, write:\n[CHAT]:: acquaintance's name\n[TIME]:: hours for chat. Select a number from 0.5, 1 and 2."
            + "\nIf {agent_name} want to publish posting to all acquaintances, write:\n[POST]:: what to post\n[TIME]:: 1"
            + "\n\n"
        )
        full_result = self._generate_reaction(observation, call_to_action_template, now)
        # print("\nTake Social Action Full Result: ", full_result)
        try:
            result, duration = full_result.split("\n")
        except:
            result = full_result
            duration = 1
        choice = result.split("::")[0]
        action = result.split("::")[1].strip()
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} took action: {result}",
                self.memory.now_key: now,
            },
        )

        if choice == "[CHAT]":
            duration = duration[9:].strip(".")
            duration = int(duration) if duration.isdigit() else 1
        else:
            duration = None

        return choice, action, duration


    def generate_dialogue_response(
        self, observation: str, now: Optional[datetime] = None
    ) -> Tuple[bool, str]:
        """React to a given observation."""
        call_to_action_template = (
            "What would {agent_name} say? To end the conversation, write:"
            ' GOODBYE: "what to say". Otherwise to continue the conversation,'
            ' write: SAY: "what to say next"\n\n'
        )
        full_result = self._generate_reaction(
            observation, call_to_action_template, now=now
        )
        result = full_result.strip().split("\n")[0]
        if "GOODBYE:" in result:
            farewell = self._clean_response(result.split("GOODBYE:")[-1])
            self.memory.save_context(
                {},
                {
                    self.memory.add_memory_key: f"{self.name} observed "
                    f"{observation} and said {farewell}",
                    self.memory.now_key: now,
                },
            )
            return False, f"{self.name} said {farewell}"
        if "SAY:" in result:
            response_text = self._clean_response(result.split("SAY:")[-1])
            self.memory.save_context(
                {},
                {
                    self.memory.add_memory_key: f"{self.name} observed "
                    f"{observation} and said {response_text}",
                    self.memory.now_key: now,
                },
            )
            return True, f"{self.name} said {response_text}"
        else:
            return False, result

    def generate_dialogue(
        self, agent2, observation: str, now: Optional[datetime] = None
    ) -> Tuple[bool, str]:
        """React to a given observation."""
        call_to_action_template = (
            "What will be said between {agent_name} and {agent_name2}? {agent_name} initiates the conversation first. Please simulate their conversation."
            "{agent_name} and {agent_name2} should not say anything about movies they have not watched or heard about."
            "Write the dialogue in the following format:"
            "[{agent_name}]:"
            "[{agent_name2}]:"
        )
        full_result = self._generate_reaction_bewteen_two(
            agent2, observation, call_to_action_template, now=now
        )
        # print("\n\n Generate dialogue full result: ", full_result)

        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} had a dialogue with {agent2.name}: {full_result} ",
                self.memory.now_key: now,
            },
        )
        agent2.memory.save_context(
            {},
            {
                agent2.memory.add_memory_key: f"{agent2.name} had a dialogue with {self.name}: {full_result} ",
                agent2.memory.now_key: now,
            },
        )
        return full_result

    def social_network_post(self, observation, now) -> str:
        """Posting for all online friends"""
        call_to_action_template = (
            "{agent_name} recently reads a book on recommender systems. "
            "If {agent_name} feelings after reading the book strongly align with {agent_name} positive behavioral preferences, "
            "{agent_name} will make a post about the book on a social platform.\n"
            "Follow this format only:\n"
            "Public Posting: [YES/NO]\n"
            "If NO, then summary: None\n"
            "If YES, then provide a summary:\n"
            "Follow those steps for writing the summary:\n"
            "- Begin with a brief personal reaction or feeling.\n"
            "- Mention the book's title and author.\n"
            "- Give a short summary or share what you liked or disliked (in 1 sentence, avoid spoilers).\n"
            "- Conclude with a recommendation or suggest who might enjoy the book.\n"
            "- Make your post sound friendly and engaging.\n"
            "Summary:[write a summary based on the given steps]"
            )
        result = self._generate_reaction(observation, call_to_action_template, now)
        # print("Social Network Post Result: ", result)
        return result

    def publish_posting(self, observation, now) -> str:
        """Publish posting to all acquaintances."""

        call_to_action_template = (
            "Posts should be related to {observation} on recommender systems. "
            "{agent_name} should not say anything about movies that have not watched or heard about."
            + "\nIf you were {agent_name}, what will you post? Respond in one line."
            + "\n\n"
        )

        result = self._generate_reaction(observation, call_to_action_template, now)
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} is publishing posting to all acquaintances. {self.name} posted {result}",
                self.memory.now_key: now,
            },
        )
        return result

    def update_watched_history(self, items, now=None):
        """Update history by the items bought. If the number of items in the history achieves the BUFFERSIZE, delete the oldest item."""
        item = []
        # self.watched_history.extend(items)
        item.extend(items)
        if self.BUFFERSIZE:
                movie_title_tokens = []
                for token in item:
                    if token == ";":
                        break
                    if token not in {'<', '>'}:
                        movie_title_tokens.append(token)

                item = ''.join(movie_title_tokens)
            # self.watched_history = self.watched_history[-self.BUFFERSIZE :]
        self.watched_history.append(item)
        # print("\nRead History: ", self.watched_history)

    def update_heared_history(self, items, now=None):
        """Update history by the items heard. If the number of items in the history achieves the BUFFERSIZE, delete the oldest item."""
        self.heared_history.extend(items)
        if len(self.heared_history) > self.BUFFERSIZE:
            self.heared_history = self.heared_history[-self.BUFFERSIZE :]
        # print("\nHeared History: ", self.heared_history)

In [ ]:
from tqdm import tqdm
import time
import concurrent.futures
# import json
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

import math
import faiss
import re
import dill
import numpy as np
from typing import List

import threading
import heapq

lock = threading.Lock()


class Simulator:
    """
    Simulator class for running the simulation.
    """

    def __init__(self, config: CfgNode, logger: logging.Logger):
        self.config = config
        self.logger = logger
        self.round_cnt = 0
        self.round_msg: List[Message] = []
        self.active_agents: List[int] = []  # active agents in current round
        self.active_agent_threshold = config["active_agent_threshold"]
        self.active_method = config["active_method"]
        # self.file_name_path: List[str] = []
        self.play_event = threading.Event()
        self.working_agents: List[RecAgent] = []  # busy agents
        self.now = datetime.now().replace(hour=8, minute=0, second=0)
        self.interval = parse_interval(config["interval"])
        # self.round_entropy = []
        self.socialnetwork = SocialNetwork()
        self.rec_cnt = [0] * config["agent_num"]
        self.hit = []
        self.ndcg = []
        self.av_hits = []
        self.av_ndcg = []
        self.pairs = []
        self.t_en = []
        self.agent_interpage = [0] * config["agent_num"]
        # self.rec_stat = RecommenderStat(
        #     tot_user_num=0,
        #     cur_user_num=0,
        #     tot_item_num=0,
        #     inter_num=0,
        #     rec_model=config["rec_model"],
        #     pop_items=[],
        # )
        self.social_stat = SocialStat(
            tot_user_num=0,
            cur_user_num=0,
            tot_link_num=0,
            chat_num=0,
            cur_chat_num=0,
            post_num=0,
            pop_items=[],
            network_density=0,
        )

    def get_file_name_path(self):
        return self.file_name_path

    def load_simulator(self):
        """Load and initiate the simulator."""
        self.round_cnt = 0
        self.data = Data(self.config)
        self.agents = self.agent_creation()
        self.recsys = Recommender(self.config, self.logger, self.data)
        # used for recommendation models
        self.recsys.create_train_data()
        self.recsys.load_best_model()
        self.recsys.get_full_rankings(use_test=True)
        self.logger.info("Simulator loaded.")

    def save(self, save_dir_name):
        """Save the simulator status of current round"""
        ensure_dir(save_dir_name)
        ID = generate_id(self.config["simulator_dir"])
        file_name = f"{ID}-Round[{self.round_cnt}]-AgentNum[{self.config['agent_num']}]-{datetime.now().strftime('%Y-%m-%d-%H_%M_%S')}"
        self.file_name_path.append(file_name)
        save_file_name = os.path.join(save_dir_name, file_name + ".pkl")
        # with open(save_file_name, "wb") as f:
        #     dill.dump(self.__dict__, f)
        self.logger.info("Current simulator Save in: \n" + str(save_file_name) + "\n")
        self.logger.info(
            "Simulator File Path (root -> node): \n" + str(self.file_name_path) + "\n"
        )
        cpkt_path = os.path.join(self.config["ckpt_path"], file_name + ".pth")
        self.recsys.save_model(cpkt_path)
        self.logger.info(
            "Current Recommender Model Save in: \n" + str(cpkt_path) + "\n"
        )

    @classmethod
    def restore(cls, restore_file_name, config, logger):
        """Restore the simulator status from the specific file"""
        with open(restore_file_name + ".pkl", "rb") as f:
            obj = cls.__new__(cls)
            obj.__dict__ = dill.load(f)
            obj.config, obj.logger = config, logger
            return obj

    def relevance_score_fn(self, score: float) -> float:
        """Return a similarity score on a scale [0, 1]."""
        # This will differ depending on a few things:
        # - the distance / similarity metric used by the VectorStore
        # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
        # This function converts the euclidean norm of normalized embeddings
        # (0 is most similar, sqrt(2) most dissimilar)
        # to a similarity function (0 to 1)
        return 1.0 - score / math.sqrt(2)

    def create_new_memory_retriever(self):
        """Create a new vector store retriever unique to the agent."""
        # Define your embedding model
        embeddings_model = OpenAIEmbeddings()
        # Initialize the vectorstore as empty
        embedding_size = 1536
        index = faiss.IndexFlatL2(embedding_size)
        vectorstore = FAISS(
            embeddings_model.embed_query,
            index,
            InMemoryDocstore({}),
            {},
            relevance_score_fn=self.relevance_score_fn,
        )

        # If choose RecAgentMemory, you must use RecAgentRetriever rather than TimeWeightedVectorStoreRetriever.
        RetrieverClass = (
            RecAgentRetriever
            if self.config["recagent_memory"] == "recagent"
            else TimeWeightedVectorStoreRetriever
        )

        return RetrieverClass(
            vectorstore=vectorstore, memory_stream=["importance"], now=self.now, k=5
        )

    def check_active(self, index: int):
        # If agent's previous action is completed, reset the event
        agent = self.agents[index]
        if isinstance(agent, RoleAgent):
            return True

        if (
            self.active_agent_threshold
            and len(self.active_agents) >= self.active_agent_threshold
        ):
            return False
        # If the book does not end, the agent continues reading the book.
        if agent.event.action_type == "reading":
            self.round_msg.append(
                Message(
                    agent_id=agent.id,
                    action="READ",
                    content=f"{agent.name} is reading book.",
                )
            )
            return False

        active_prob = agent.get_active_prob(self.active_method)
        if np.random.random() > active_prob:
            agent.no_action_round += 1
            return False
        self.active_agents.append(index)
        return True

    def pause(self):
        self.play_event.clear()

    def play(self):
        self.play_event.set()

    def one_step(self, agent_id):
        """Run one step of an agent."""
        self.play_event.wait()
        if not self.check_active(agent_id):
            # print("Not Active")
            return [
                Message(agent_id=agent_id, action="NO_ACTION", content="No action.")
            ]
        agent = self.agents[agent_id]
        name = agent.name
        message = []
        page = 0
        book_rating = None

        choice, observation = agent.take_action(self.now)

        with lock:
            heapq.heappush(self.working_agents, agent)
        if "RECOMMENDER" in choice:
            ids = []
            self.rec_cnt[agent_id] += 1
            self.logger.info(f"{name} enters the recommender system.")
            message.append(
                Message(
                    agent_id=agent_id,
                    action="RECOMMENDER",
                    content=f"{name} enters the recommender system.",
                )
            )
            self.round_msg.append(
                Message(
                    agent_id=agent_id,
                    action="RECOMMENDER",
                    content=f"{name} enters the recommender system.",
                )
            )


            leave = False

            # choice the ratio of ground truth items and random items
            # gt_ratio, rd_ratio, total_items = 1, 1, 10
            # item_ids, rec_items, gt_items = self.recsys.get_full_manual_items(agent_id, gt_ratio, rd_ratio, total_items)

            # from recommender system (MF, NueMF, LightGCN, MultiVAE)
            # self.recsys.get_full_rankings() # Is needed!

            # without continual page
            item_ids = self.recsys.full_rankings[agent_id][page*self.recsys.page_size:(page+1)*self.recsys.page_size]
            # start_idx = self.agent_interpage[agent_id] # continual page
            # end_idx = start_idx + self.recsys.page_size # continual page
            # item_ids = self.recsys.full_rankings[agent_id][start_idx:end_idx] # continual page
            # print("Item IDs: ", item_ids)

            rec_items = self.recsys.get_rec_discription(item_ids)
            self.recsys.update_user_interactions(agent_id, item_ids)
            # item_ids, rec_items = self.recsys.get_full_sort_items(agent_id)
            cnt = 0
            searched_name = None
            while not leave:
                self.logger.info(
                          f"{name} is recommended {rec_items}."
                )
                #     f"{name} is recommended {rec_items[page*self.recsys.page_size:(page+1)*self.recsys.page_size]}."
                # )
                message.append(
                    Message(
                        agent_id=agent_id,
                        action="RECOMMENDER",
                        # content=f"{name} is recommended {rec_items[page*self.recsys.page_size:(page+1)*self.recsys.page_size]}.",
                        content=f"{name} is recommended {rec_items}.",
                    )
                )
                self.round_msg.append(
                    Message(
                        agent_id=agent_id,
                        action="RECOMMENDER",
                        # content=f"{name} is recommended {rec_items[page*self.recsys.page_size:(page+1)*self.recsys.page_size]}.",
                        content=f"{name} is recommended {rec_items}.",
                    )
                )
                observation = f"{name} is browsing the recommender system."
                if searched_name is not None:
                    observation = (
                        observation
                        # + f" {name} has searched for {searched_name} in recommender system and recommender system returns item list:{rec_items[page*self.recsys.page_size:(page+1)*self.recsys.page_size]} as search results."
                        + f" {name} has searched for {searched_name} in recommender system and recommender system returns item list:{self.recsys.data.search_items(searched_name, k=5)} as search results."
                    )
                else:
                    observation = (
                        observation
                        + f" {name} is recommended {rec_items}."
                    )
                while True:
                    # print("Observation: ", observation)
                    choice, action = agent.take_recommender_action(observation, self.now)
                    ids.extend(item_ids)


                    if "BUY" in choice and (
                        agent.event.action_type == "idle"
                        or agent.event.action_type == "posting"
                        ):
                       page += 1
                      #  self.agent_interpage[agent_id] = start_idx + 1 # continual page
                       # item_names = rec_items[page * self.recsys.page_size + action - 1]
                       item_names = rec_items[action - 1]
                       # print("\n\nItem Names: ", item_names, "\n")
                       # item_id = item_ids[page * self.recsys.page_size + action - 1]
                       item_id = item_ids[action - 1]
                       duration = 2
                       agent.event = update_event(
                           original_event=agent.event,
                           start_time=self.now,
                           duration=duration,
                           target_agent=None,
                           action_type="reading",
                       )

                       self.logger.info(f"{name} reads {item_names}")
                       message.append(
                           Message(
                               agent_id=agent_id,
                               action="RECOMMENDER",
                               content=f"{name} reads {item_names}.",
                           )
                        )
                       self.round_msg.append(
                           Message(
                              agent_id=agent_id,
                              action="RECOMMENDER",
                              content=f"{name} reads {item_names}.",
                            )
                        )
                       agent.update_watched_history(item_names)
                       self.recsys.update_history_by_id(
                           agent_id,
                           [item_id],
                           )


                       self.hit.append([self.recsys.data.hit_at_k([item_id], ids, a) for a in range(1, 11)])
                       self.ndcg.append([self.recsys.data.ndcg_at_k([item_id], ids, b) for b in range(1, 11)])

                       self.recsys.update_positive_by_id(agent_id, item_id)

                       item_descriptions = self.data.get_item_description_by_name(
                           [item_names]
                           )
                       if len(item_descriptions) == 0:
                          self.logger.info(f"{name} leaves the recommender system.")
                          message.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"{name} leaves the recommender system.",
                            )
                            )
                          self.round_msg.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"{name} leaves the recommender system.",
                            )
                            )
                          leave = True
                          continue

                       observation = f"{name} has just finished reading {item_names};;{item_descriptions[0]}."
                       feelings = agent.generate_feeling(
                            observation, self.now + timedelta(hours=duration)
                            )
                      #  # Social network post analysis
                      #  public_post = (
                      #       observation
                      #       + f"Feelings after reading the book:"
                      #       + feelings)
                      #  social_network_post = agent.social_network_post(
                      #       public_post, self.now + timedelta(hours=duration)
                      #       )
                      #  yes_match = re.search(r'Public Posting:\s*(YES|NO)', social_network_post)
                      #  public_posting = yes_match.group(1) if yes_match else None

                      #  # Extract summary only if YES
                      #  summary = None
                      #  if public_posting == "YES":
                      #     summary_match = re.search(r'Summary:\s*(.+)', social_network_post, re.DOTALL)
                      #     summary = summary_match.group(1).strip() if summary_match else None
                      #     book_genre = self.recsys.data.get_genres_by_id([item_id])
                      #     self.socialnetwork.create_post(agent_id, self.round_cnt, public_posting, summary, book_genre)
                      #  # End analysis of social network post

                       self.logger.info(f"{name} feels: {feelings}")

                       message.append(
                            Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} feels: {feelings}",
                            )
                            )
                       self.round_msg.append(
                            Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} feels: {feelings}",
                            )
                            )
                       # print("Item ID: ", item_id)
                       historical_rating = self.recsys.data.user_avg_rating[agent_id]
                       # print("Historical Rating: ", historical_rating)
                       user_profile = self.recsys.data.get_user_profile(agent_id)
                       # print("User Profile: ", user_profile)
                       feel_rating = (
                            user_profile
                            + feelings
                            + f" {name} has a historical rating:{historical_rating}."
                            )
                       book_rating = agent.generate_rec_rating(feel_rating, self.now)
                       self.recsys.add_review(agent_id, book_rating, feelings)
                       self.pairs.append(self.recsys.calculation_of_rating(agent_id, item_names, book_rating))
                       searched_name = None
                       leave = True
                       break

                    elif "NEXT" in choice:
                        # self.agent_interpage[agent_id] = start_idx + 1 # continual page
                        self.logger.info(f"{name} looks next page.")
                        page += 1
                        # without continual page
                        nitem_ids = self.recsys.full_rankings[agent_id][page*self.recsys.page_size:(page+1)*self.recsys.page_size]

                        nrec_items = self.recsys.get_rec_discription(item_ids)

                        # nitem_ids, nrec_items, ngt_items = self.recsys.get_full_manual_items(agent_id, gt_ratio, rd_ratio, total_items)
                        for i in range(len(nitem_ids)):
                            self.recsys.add_train_data(
                                agent_id, nitem_ids, 0
                                )

                        message.append(
                            Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} looks next page.",
                            )
                            )
                        self.round_msg.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"{name} looks next page.",
                                )
                            )
                        rec_items = nrec_items
                        item_ids = nitem_ids
                        # Update observation for new recommendations and loop again (do not break)
                        observation = f"{name} is browsing the recommender system and looks next page. {name} is recommended {rec_items}."

                        if page == 5:
                           page = 0
                           self.logger.info(f"{name} scroll maximum pages.")
                           self.logger.info(f"{name} leaves the recommender system.")
                           message.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"{name} scroll maximum pages. {name} leaves the recommender system.",
                            )
                            )
                           self.round_msg.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"No more items. {name} leaves the recommender system.",
                            )
                            )
                           for post in self.socialnetwork.get_ranked_posts_for_agent(agent_id):
                               print(self.socialnetwork.format_post(post))
                           leave = True
                           break
                        else:
                            continue
                    elif "SEARCH" in choice:
                        # self.agent_interpage[agent_id] = start_idx + 1 # continual page
                        page += 1
                        observation = f"{name} is searching in recommender system."
                        item_name = agent.search_item(observation, self.now)
                        self.logger.info(f"{name} searches {item_name}.")
                        message.append(
                            Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} searches {item_name}.",
                        )
                            )
                        self.round_msg.append(
                            Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} searches {item_name}.",
                        )
                            )
                        item_names = extract_item_names(item_name)
                        if item_names == []:
                            agent.memory.add_memory(
                            f"There are no items related in the system.", now=self.now
                            )
                            self.logger.info("There are no related items in the system.")
                            message.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"There are no related products in the system.",
                            )
                            )
                            self.round_msg.append(
                                Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"There are no related products in the system.",
                            )
                                )
                            leave = True
                            continue
                        item_name = item_names[0]
                        search_items = self.recsys.get_search_items(item_name)
                        self.logger.info(f"Recommender returned {search_items}.")
                        message.append(
                            Message(
                                agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"Recommender returned {search_items}.",
                        )
                            )
                        self.round_msg.append(
                            Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"Recommender returned {search_items}.",
                        )
                            )
                        if len(search_items) != 0:
                           rec_items = search_items
                           item_ids = self.data.get_item_ids_exact(search_items)
                           page = 0
                           searched_name = item_name
                        else:
                           agent.memory.add_memory(
                            f"There are no items related to {item_name} in the system.",
                            now=self.now,
                            )
                           self.logger.info("There are no related items in the system.")
                           message.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"There are no related products in the system.",
                            )
                            )
                           self.round_msg.append(
                            Message(
                                agent_id=agent_id,
                                action="RECOMMENDER",
                                content=f"There are no related products in the system.",
                            )
                            )
                        break
                else:
                    self.logger.info(f"{name} leaves the recommender system.")
                    message.append(
                        Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} leaves the recommender system.",
                        )
                    )
                    self.round_msg.append(
                        Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} leaves the recommender system.",
                        )
                    )
                    leave = True
                cnt += 1
                if cnt == 5:
                    self.logger.info(f"{name} leaves the recommender system.")
                    message.append(
                        Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} leaves the recommender system.",
                        )
                    )
                    self.round_msg.append(
                        Message(
                            agent_id=agent_id,
                            action="RECOMMENDER",
                            content=f"{name} leaves the recommender system.",
                        )
                    )
                    leave = True
            N_expose = page * self.config['page_size']
            N_view = 1 if book_rating is not None else 0
            N_like = 1 if book_rating is not None and int(book_rating) > 3 else 0
            if page == 0:
                N_exit = 5
            else:
                N_exit = page
            S_sat = 5
            self.recsys.add_user(agent_id, N_expose, N_view, N_like, N_exit, S_sat)
            # entropy = get_entropy(ids, self.data)
            # self.round_entropy.append(entropy)
            # self.recsys.round_record[agent_id].append(ids)

        elif "SOCIAL" in choice:
            contact = self.data.get_all_contacts(agent_id)
            # print("\n\nAll contacts in SOCIAL Media: ", contacts)
            num_agent = ['Emily Johnson', 'Sarah Johnson', 'David Miller', 'Sarah Garcia', ' John Smith', 'Emily Miller', 'Sarah Smith', 'William Brown', 'John Davis', 'Christopher Wilson']
            # num_agent = ['David Smith', 'David Miller', 'James Brown', 'Sarah Miller', 'John Taylor', 'Sarah Williams', 'James Jones', 'Jane Brown', 'David Jones', 'James Brown']
            for ng in num_agent:
                contacts = [con for con in contact if con == ng]
            if len(contacts) == 0:
                self.logger.info(f"{name} has no acquaintance.")
                message.append(
                    Message(
                        agent_id=agent_id,
                        action="SOCIAL",
                        content=f"{name} has no acquaintance.",
                    )
                )
                self.round_msg.append(
                    Message(
                        agent_id=agent_id,
                        action="SOCIAL",
                        content=f"{name} has no acquaintance.",
                    )
                )
            else:
                self.social_stat.cur_user_num += 1
                self.logger.info(f"{name} is going to social media.")
                message.append(
                    Message(
                        agent_id=agent_id,
                        action="SOCIAL",
                        content=f"{name} is going to social media.",
                    )
                )
                self.round_msg.append(
                    Message(
                        agent_id=agent_id,
                        action="SOCIAL",
                        content=f"{name} is going to social media.",
                    )
                )
                observation = f"{name} is going to social media. {name} and {contacts} are acquaintances. {name} can chat with acquaintances, or post to all acquaintances. What will {name} do?"
                choice, action, duration = agent.take_social_action(
                    observation, self.now
                )
                print("\nSocial Media Choice: ",choice)
                print("\nSocial Media Action: ",action)
                if "CHAT" in choice:
                    agent_name2 = action.strip(" \t\n'\"")
                    # print("agent_name2:", agent_name2)
                    agent_id2 = self.data.get_user_ids([agent_name2])[0]
                    agent2 = self.agents[agent_id2]
                    # If agent2 is watching moives, he cannot be interupted.
                    if agent2.event.action_type == "reading":
                        print("\n\nWatching Movie...........\n")
                        agent.memory.add_memory(
                            f"{agent.name} wants to chat with {agent_name2}, but {agent_name2} is reading. So {agent.name} does nothing.",
                            now=self.now,
                        )
                        self.logger.info(
                            f"{name} wants to chat with {agent_name2}, but {agent_name2} is reading. So {name} does nothing."
                        )
                        message.append(
                            Message(
                                agent_id=agent_id,
                                action="LEAVE",
                                content=f"{name} wants to chat with {agent_name2}, but {agent_name2} is reading. So {name} does nothing.",
                            )
                        )
                        self.round_msg.append(
                            Message(
                                agent_id=agent_id,
                                action="LEAVE",
                                content=f"{name} wants to chat with {agent_name2}, but {agent_name2} is reading. So {name} does nothing.",
                            )
                        )
                        print("\nSocial Message: ", message)
                        return message

                    #  If agent2 is chatting with agent1, skipping this round
                    if is_chatting(agent, agent2):
                        print("\n\nChatting with Someone...........\n")
                        self.logger.info(f"{name} is chatting with {agent_name2}")
                        message.append(
                            Message(
                                agent_id=agent_id,
                                action="CHAT",
                                content=f"{name} is chatting with {agent_name2}",
                            )
                        )
                        self.round_msg.append(
                            Message(
                                agent_id=agent_id,
                                action="CHAT",
                                content=f"{name} is chatting with {agent_name2}.",
                            )
                        )
                        print("\n\nIs Chatting:", message)
                        return message
                    agent.event = update_event(
                        original_event=agent.event,
                        start_time=self.now,
                        duration=duration,
                        target_agent=agent_name2,
                        action_type="chatting",
                    )
                    agent2.event = update_event(
                        original_event=agent2.event,
                        start_time=self.now,
                        duration=duration,
                        target_agent=name,
                        action_type="chatting",
                    )
                    self.logger.info(f"{name} is chatting with {agent_name2}.")
                    self.social_stat.chat_num += 1
                    message.append(
                        Message(
                            agent_id=agent_id,
                            action="CHAT",
                            content=f"{name} is chatting with {agent_name2}.",
                        )
                    )
                    self.round_msg.append(
                        Message(
                            agent_id=agent_id,
                            action="CHAT",
                            content=f"{name} is chatting with {agent_name2}.",
                        )
                    )
                    # If the system has a role, and it is her term now.
                    if self.config["play_role"] and self.data.role_id == agent_id:
                        print("\n\nPlay Role...........\n")
                        conversation = ""
                        observation = f"{name} is going to chat with {agent2.name}."
                        # Obtain the response from the role.
                        contin, result, role_dia = agent.generate_role_dialogue(
                            agent2, observation
                        )
                        conversation += role_dia + result
                        self.logger.info(role_dia)
                        self.logger.info(result)
                        # If both of them do not stop, an extra round will be held.
                        while contin:
                            contin, result, role_dia = agent.generate_role_dialogue(
                                agent2, observation, conversation
                            )
                            conversation += role_dia + result
                            self.logger.info(role_dia)
                            self.logger.info(result)
                    else:
                        print("\n\nLast Else...........\n")
                        observation = f"{name} is going to chat with {agent2.name}."
                        # If an agent wants to chat with the role.
                        if self.config["play_role"] and agent_id2 == self.data.role_id:
                            print("\nEntering IF play role")
                            conversation = ""
                            observation = f"{name} is going to chat with {agent2.name}."
                            # Obtain the response from the agent(LLM).
                            contin, result = agent.generate_dialogue_response(
                                observation
                            )
                            agent_dia = "%s %s" % (agent.name, result)
                            self.logger.info(agent_dia)
                            # Obtain the response from the role.
                            role_contin, role_dia = agent2.generate_dialogue_response(
                                observation
                            )
                            self.logger.info(role_dia)
                            contin = contin and role_contin
                            conversation += agent_dia + role_dia
                            # If both of them do not stop, an extra round will be held.
                            while contin:
                                observation = (
                                    f"{name} is going to chat with {agent2.name}."
                                )
                                contin, result = agent.generate_dialogue_response(
                                    observation
                                )
                                agent_dia = "%s %s" % (agent.name, result)
                                self.logger.info(agent_dia)
                                (
                                    role_contin,
                                    role_dia,
                                ) = agent2.generate_dialogue_response(observation)
                                self.logger.info(role_dia)
                                contin = contin and role_contin
                                conversation += agent_dia + role_dia
                        else:
                            # Otherwise, two agents(LLM) will generate dialogues.
                            print("\nEntering Else....:")
                            conversation = agent.generate_dialogue(agent2, observation)
                            # print("\n Conversation: ", conversation)
                        self.logger.info(conversation)

                    msgs = []
                    matches = re.findall(r"\[([^]]+)\]:\s*(.*)", conversation)
                    print("\n\n Matches..: ", matches)
                    for match in matches:
                        speaker = match[0]
                        content = match[1]
                        if speaker == agent.name:
                            id = agent_id
                            id2 = agent_id2
                        else:
                            id = agent_id2
                            id2 = agent_id
                        item_names = extract_item_names(content, "SOCIAL")
                        self.data.add_mention_cnt(item_names)
                        if item_names != []:
                            self.agents[id2].update_heared_history(item_names)
                            # New lines
                            aitem_id = self.recsys.data.get_item_ids(item_names)
                            self.recsys.update_positive_by_id(agent_id, aitem_id)
                        msgs.append(
                            Message(
                                agent_id=id,
                                action="CHAT",
                                content=f"{speaker} says:{content}",
                            )
                        )
                        self.round_msg.append(
                            Message(
                                agent_id=id,
                                action="CHAT",
                                content=f"{speaker} says:{content}",
                            )
                        )
                    message.extend(msgs)

                else:
                    print("\n\nPublic Posting")
                    self.social_stat.post_num += 1
                    self.logger.info(f"{name} is posting.")
                    observation = f"{name} want to post for all acquaintances."
                    observation = agent.publish_posting(observation, self.now)
                    item_names = extract_item_names(observation, "SOCIAL")
                    self.logger.info(agent.name + " posted: " + observation)
                    if agent.event.action_type == "idle":
                        agent.event = update_event(
                            original_event=agent.event,
                            start_time=self.now,
                            duration=0.1,
                            target_agent=None,
                            action_type="posting",
                        )
                    message.append(
                        Message(
                            agent_id=agent_id,
                            action="POST",
                            content=agent.name + " posts: " + observation,
                        )
                    )
                    self.round_msg.append(
                        Message(
                            agent_id=agent_id,
                            action="POST",
                            content=agent.name + " posts: " + observation,
                        )
                    )
                    for i in self.agents.keys():
                        if self.agents[i].name in contacts:
                            self.agents[i].memory.add_memory(
                                agent.name + " posts: " + observation, now=self.now
                            )
                            self.agents[i].update_heared_history(item_names)
                            # New lines
                            aitem_id = self.recsys.data.get_item_ids(item_names)
                            self.recsys.update_positive_by_id(agent_id, aitem_id)

                            message.append(
                                Message(
                                    agent_id=self.agents[i].id,
                                    action="POST",
                                    content=self.agents[i].name
                                    + " observes that"
                                    + agent.name
                                    + " posts: "
                                    + observation,
                                )
                            )
                            self.round_msg.append(
                                Message(
                                    agent_id=self.agents[i].id,
                                    action="POST",
                                    content=self.agents[i].name
                                    + " observes that"
                                    + agent.name
                                    + " posts: "
                                    + observation,
                                )
                            )

                    self.logger.info(f"{contacts} get this post.")

        else:
            self.logger.info(f"{name} does nothing.")
            message.append(
                Message(
                    agent_id=agent_id, action="LEAVE", content=f"{name} does nothing."
                )
            )
            self.round_msg.append(
                Message(
                    agent_id=agent_id, action="LEAVE", content=f"{name} does nothing."
                )
            )
        # print("\nFinal Message: ", message, "\n")

        return message

    def update_working_agents(self):
        with lock:
            agent: RecAgent = None
            while len(self.working_agents) > 0:
                agent = heapq.heappop(self.working_agents)
                if agent.event.end_time <= self.now:
                    agent.event = reset_event(self.now)
                else:
                    break
            if agent is not None and agent.event.end_time > self.now:
                heapq.heappush(self.working_agents, agent)

    def round(self):
        """
        Run one step for all agents.
        """
        messages = []
        futures = []
        precisions = []
        recalls = []
        accuracies = []
        f1s = []
        self.hit = []
        self.ndcg = []
        read = []
        hear = []

        # The user's role takes one step first.
        if self.config["play_role"]:
            role_msg = self.one_step(self.data.role_id)
            messages.extend(role_msg)
            # print("Enter in the play_role: ", messages)

        if self.config["execution_mode"] == "parallel":
            with concurrent.futures.ThreadPoolExecutor() as executor:
                for i in tqdm(range(1, self.config["agent_num"])):
                    futures.append(executor.submit(self.one_step, i))
                    # print("one step stopped..................", futures)
                    # time.sleep(10)

            for future in concurrent.futures.as_completed(futures):
                # try:
                msgs = future.result()
                messages.append(msgs)
                #     print(f"Result: {messages}")
                # except Exception as e:
                #     print(f"Exception: {e}")

        else:
            for i in tqdm(range(1, self.config["agent_num"])):
                msgs = self.one_step(i)
                messages.append(msgs)
        self.now = add_interval(self.now, self.interval)

        for i, agent in self.agents.items():
            agent.memory.update_now(self.now)
            # Comment this line whenever you don't want to use the memory
            agent.memory.longTermMemory.print_memory()
            # print("\n")
            read_books = self.recsys.data.get_item_ids(agent.watched_history)

            item_set = self.recsys.interaction_dict.get(agent.id, set())
            all_item = list(item_set)
            precision, recall, accuracy, f1 = self.recsys.calculate_user_metrics(agent.id, read_books, all_item)
            precisions.append(precision)
            recalls.append(recall)
            accuracies.append(accuracy)
            f1s.append(f1)
            read_list = self.recsys.data.get_item_ids(agent.watched_history)
            hear_list = self.recsys.data.get_item_ids(agent.heared_history)
            read.append(get_entropy(read_list, self.recsys.data))
            hear.append(get_entropy(hear_list, self.recsys.data))

            if self.round_cnt % 16 == 0:
               for k in [1, 3, 5, 10]:
                   precisonk, recallk = self.recsys.precisionandrecallk(agent.id, read_books, k)
                   print(f"Precision@{k}: {precisonk}, Recall@{k}: {recallk}")




        avg_precision = sum(precisions) / len(precisions) if precisions else 0
        avg_recall = sum(recalls) / len(recalls) if recalls else 0
        avg_accuracy = sum(accuracies) / len(accuracies) if accuracies else 0
        avg_f1 = sum(f1s) / len(f1s) if f1s else 0

        mean_hits = np.mean(np.array(self.hit), axis=0)
        mean_ndcg = np.mean(np.array(self.ndcg), axis=0)
        self.av_hits.append(mean_hits)
        self.av_ndcg.append(mean_ndcg)

        print(" ".join([f"HIT@{k}: {mean_hits[k-1]:.4f}" for k in range(1, 11)]))
        print(" ".join([f"NDCG@{k}: {mean_ndcg[k-1]:.4f}" for k in range(1, 11)]))
        print(" ".join([f"Average HIT@{k}: {np.mean(self.av_hits, axis=0)[k-1]:.4f}" for k in range(1, 11)]))
        print(" ".join([f"Average NDCG@{k}: {np.mean(self.av_ndcg, axis=0)[k-1]:.4f}" for k in range(1, 11)]))
        print("Average Precision: ", avg_precision, "Average Recall: ", avg_recall, "Average Accuracy:", avg_accuracy,
              "Average F1-Score: ", avg_f1)
        print("Entropy: ", self.recsys.get_entropy())
        print("Read Entropy: ", np.mean(read))
        print("Hear Entropy: ", np.mean(hear))
        print("Total Entropy: ", np.mean(read) + np.mean(hear))
        self.t_en.append(np.mean(read) + np.mean(hear))
        if self.round_cnt == 20:
          print("Total Entropys: ", self.t_en)
          df = pd.DataFrame(self.recsys.rating_feeling)
          df.to_csv('/content/drive/MyDrive/S4065511/data/rating_feeling.csv', index=False)
        print("Length of user rating pairs: ", len(self.pairs))
        mse, rmse, gt_pct, pred_pct, loglike, ob_loglike, spearman = self.recsys.calc_mse_rmse_rating_percentages(self.pairs)
        metrics = self.recsys.satisfaction_metrics()
        print("Total MSE: ", mse)
        print("Total RMSE: ", rmse)
        print("Total Log Likelihood: ", loglike)
        print("Total per Observe Log Likelihood: ", ob_loglike)
        print("Total Ground Truth Rating Percentages: ", gt_pct)
        print("Total Predicted Rating Percentages: ", pred_pct)
        print("Total Spearman Correlation: ", spearman)
        print("Total Satisfaction Metrics (Pview, Nlike, Plike, Nexit, Ssat): ", metrics)

        self.update_working_agents()
        return messages

    def create_agent(self, i, api_key) -> RecAgent:
        """
        Create an agent with the given id.
        """
        LLM = get_llm(config=self.config, logger=self.logger, api_key=api_key)
        MemoryClass = (
            RecAgentMemory
            if self.config["recagent_memory"] == "recagent"
            else GenerativeAgentMemory
        )

        agent_memory = MemoryClass(
            llm=LLM,
            memory_retriever=self.create_new_memory_retriever(),
            now=self.now,
            verbose=False,
            reflection_threshold=10,
        )
        agent = RecAgent(
            id=i,
            name=self.data.users[i]["name"],
            age=self.data.users[i]["age"],
            gender=self.data.users[i]["gender"],
            traits=self.data.users[i]["traits"],
            status=self.data.users[i]["status"],
            interest=self.data.users[i]["interest"],
            # positive_behavior=self.data.users[i]["positive_behavior"],
            # negative_behavior=self.data.users[i]["negative_behavior"],
            relationships=self.data.get_relationship_names(i),
            feature=get_feature_description(self.data.users[i]["feature"]),
            memory_retriever=self.create_new_memory_retriever(),
            llm=LLM,
            memory=agent_memory,
            event=reset_event(self.now),
            avatar_url=get_avatar_url(
                id=i, gender=self.data.users[i]["gender"], type="origin"
            ),
            idle_url=get_avatar_url(
                id=i, gender=self.data.users[i]["gender"], type="idle"
            ),
            watching_url=get_avatar_url(
                id=i, gender=self.data.users[i]["gender"], type="watching"
            ),
            chatting_url=get_avatar_url(
                id=i, gender=self.data.users[i]["gender"], type="chatting"
            ),
            posting_url=get_avatar_url(
                id=i, gender=self.data.users[i]["gender"], type="posting"
            ),
        )
        # observations = self.data.users[i]["observations"].strip(".").split(".")
        # for observation in observations:
        #     agent.memory.add_memory(observation, now=self.now)
        return agent


    def agent_creation(self):
        """
        Create agents in parallel
        """
        agents = {}
        api_keys = list(self.config["api_keys"])
        agent_num = int(self.config["agent_num"])
        # Add ONE user controllable user into the simulator if the flag is true.
        # We block the main thread when the user is creating the role.
        if self.config["play_role"]:
            role_id = self.data.get_user_num()
            api_key = api_keys[role_id % len(api_keys)]
            agent = self.create_user_role(role_id, api_key)
            agents[role_id] = agent
            self.data.role_id = role_id
        if self.active_method == "random":
            active_probs = [self.config["active_prob"]] * agent_num
        else:
            active_probs = np.random.pareto(self.config["active_prob"] * 10, agent_num)
            active_probs = active_probs / active_probs.max()

        if self.config["execution_mode"] == "parallel":
            futures = []
            start_time = time.time()
            with concurrent.futures.ThreadPoolExecutor() as executor:
                for i in range(1, agent_num):
                    api_key = api_keys[i % len(api_keys)]
                    futures.append(executor.submit(self.create_agent, i, api_key))
                for future in tqdm(concurrent.futures.as_completed(futures)):
                    agent = future.result()
                    agent.active_prob = active_probs[agent.id]
                    agents[agent.id] = agent
                    self.socialnetwork.add_agent(agent.id, agent.name, agent.interest)
                    print(f"Agent Creation {agent.id}: {agent}")
            end_time = time.time()
            self.logger.info(
                f"Time for creating {agent_num} agents: {end_time-start_time}"
            )
        else:
            for i in tqdm(range(1, agent_num)):
                api_key = api_keys[i % len(api_keys)]
                agent = self.create_agent(i, api_key)
                agent.active_prob = active_probs[agent.id]
                agents[agent.id] = agent

        return agents